In [1]:
# import os
# import shutil
# import pandas as pd

# # Paths — these folders should be inside "Documents"
# input_folder = 'Hackathon_dataset_1'
# sorted_folder = 'HSort1'
# unsorted_folder = 'HUnsorted'

# # Start and end date patterns to accept
# start_options = ["01-01-2010", "2010-01-01"]
# end_options = ["31-03-2023", "2023-03-31"]

# # Get list of CSV files in the input folder
# csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# for file in csv_files:
#     file_path = os.path.join(input_folder, file)

#     try:
#         # Count total number of rows
#         with open(file_path, 'r', encoding='utf-8') as f:
#             total_rows = sum(1 for _ in f)

#         # Read first and last rows from column 0
#         first_row = pd.read_csv(file_path, nrows=1, usecols=[0], encoding='utf-8')
#         last_row = pd.read_csv(file_path, skiprows=range(1, total_rows - 1), usecols=[0], encoding='utf-8')

#         # Get raw strings
#         first_val = str(first_row.iloc[0, 0]).strip()
#         last_val = str(last_row.iloc[0, 0]).strip()

#         # Compare using simple prefix match
#         if any(first_val.startswith(opt) for opt in start_options) and \
#            any(last_val.startswith(opt) for opt in end_options):
#             shutil.move(file_path, os.path.join(sorted_folder, file))
#         else:
    #         shutil.move(file_path, os.path.join(unsorted_folder, file))

    # except Exception as e:
    #     print(f"Error in {file}: {e}")
    #     shutil.move(file_path, os.path.join(unsorted_folder, file))


In [2]:
# import os
# import shutil
# import pandas as pd

# # Folder paths
# input_folder = 'Hackathon_dataset_2'
# not_needed_folder = 'not_needed'

# # List all CSV files in the data folder
# csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# for file in csv_files:
#     file_path = os.path.join(input_folder, file)

#     try:
#         # Read the first row from the file (first column only)
#         first_row = pd.read_csv(file_path, nrows=1, usecols=[0], encoding='utf-8')
#         first_value = str(first_row.iloc[0, 0]).strip()

#         # Extract year from beginning of string
#         # Works for both formats: YYYY-MM-DD and DD-MM-YYYY
#         year_part = first_value[:4]
#         if not year_part.isdigit():  # In case of DD-MM-YYYY
#             year_part = first_value[-4:]

#         start_year = int(year_part)

#         # If file starts in 2023, move it to not_needed
#         if start_year == 2023:
#             shutil.move(file_path, os.path.join(not_needed_folder, file))

#     except Exception as e:
#         print(f"Error processing {file}: {e}")


In [3]:
# import pandas as pd
# import os

# # Input file (test file)
# input_file = 'Cleaned_row/AP004.csv'  # your actual file
# filename_base = os.path.splitext(os.path.basename(input_file))[0]
# output_file = os.path.join('Cleaned_row', f"{filename_base}_Cleaned.csv")

# # Helper to extract year from datetime string
# def extract_year(date_str):
#     date_str = str(date_str).strip()
#     if date_str[:4].isdigit():
#         return int(date_str[:4])  # YYYY-MM-DD
#     elif date_str[-4:].isdigit():
#         return int(date_str[-4:])  # DD-MM-YYYY
#     return None

# # Get file's start year
# first_row = pd.read_csv(input_file, nrows=1, usecols=[0], encoding='utf-8')
# first_year = extract_year(first_row.iloc[0, 0])
# print(f"File starts in year: {first_year}")

# # A) Starts before 2021 → skip rows up to 2021
# if first_year < 2021:
#     print("Skipping rows until 2021 and reading only relevant block...")

#     col_iter = pd.read_csv(input_file, usecols=[0], iterator=True, chunksize=100000, encoding='utf-8')
#     start_dates = pd.concat(col_iter)
#     start_dates['year'] = start_dates.iloc[:, 0].apply(extract_year)

#     start_2021 = start_dates[start_dates['year'] == 2021].index.min()
#     start_2023 = start_dates[start_dates['year'] == 2023].index.min()

#     if pd.isna(start_2021):
#         print("No 2021 data found.")
#         exit()
#     if pd.isna(start_2023):
#         start_2023 = len(start_dates)

#     nrows = int(start_2023 - start_2021)
#     df = pd.read_csv(input_file, skiprows=range(1, start_2021 + 1), nrows=nrows, encoding='utf-8')

# # B) Starts in 2021 or 2022 → just filter
# else:
#     print("Reading file and filtering 2021–2022...")
#     df = pd.read_csv(input_file, encoding='utf-8')
#     df['year'] = df.iloc[:, 0].apply(extract_year)
#     df = df[df['year'].isin([2021, 2022])]

# # Save output with new name
# df.drop(columns=['year'], inplace=True, errors='ignore')
# df.to_csv(output_file, index=False)
# print(f"Saved cleaned file as: {output_file}")


In [4]:
# import pandas as pd
# import os
# from datetime import datetime
# from collections import Counter

# # Folder containing the CSV files
# input_folder = "Hackathon_dataset_2"

# # Valid year range
# valid_years = list(range(2010, 2024))  # 2010 to 2023 inclusive

# # Robust timestamp parser
# def parse_timestamp(val):
#     if pd.isna(val): return pd.NaT
#     val = str(val).strip()
#     ts = pd.to_datetime(val, errors='coerce', dayfirst=False)
#     if pd.notna(ts): return ts
#     for fmt in ("%d-%m-%Y", "%d-%m-%Y %H:%M:%S", "%Y-%m-%d", "%Y/%m/%d", "%m/%d/%Y"):
#         try:
#             return datetime.strptime(val, fmt)
#         except:
#             continue
#     return pd.NaT

# # Counter for start years
# year_counter = Counter()

# # Process files
# csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# for file in csv_files:
#     try:
#         input_path = os.path.join(input_folder, file)
#         val = pd.read_csv(input_path, usecols=[0], nrows=1, encoding='utf-8').iloc[0, 0]
#         timestamp = parse_timestamp(val)
#         if pd.notna(timestamp):
#             year = timestamp.year
#             if year in valid_years:
#                 year_counter[year] += 1
#     except Exception:
#         continue  # silently skip problematic files

# # Print summary
# print("\n📊 Start Year Distribution (2010–2023):")
# for year in sorted(valid_years):
#     count = year_counter.get(year, 0)
#     print(f"{year}: {count} file(s)")

# import os

# input_folder = "Hackathon_dataset_2"

# # Count .csv files
# csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
# total_count = len(csv_files)

# print(f"\n📁 Total CSV files in '{input_folder}': {total_count}")



In [5]:
# import pandas as pd
# import os

# # Input and output folder
# input_folder = 'Cleaned_row'
# output_folder = 'cleaned2'
# os.makedirs(output_folder, exist_ok=True)

# # Helper to extract year from date strings
# def extract_year(date_str):
#     date_str = str(date_str).strip()
#     if date_str[:4].isdigit():
#         return int(date_str[:4])  # YYYY-MM-DD
#     elif date_str[-4:].isdigit():
#         return int(date_str[-4:])  # DD-MM-YYYY
#     return None

# # Loop over files in the 'cleaned' folder
# for file in os.listdir(input_folder):
#     if file.endswith('.csv'):
#         input_path = os.path.join(input_folder, file)
#         base_name = os.path.splitext(file)[0]
#         # Strip old suffixes like "_Cleaned" or "_Cleaned_Cleaned"
#         base_simple = base_name.replace("_Cleaned", "").replace("_Cleaned2", "")
#         output_path = os.path.join(output_folder, f"{base_simple}_C2.csv")


#         try:
#             # Read the CSV
#             df = pd.read_csv(input_path, encoding='utf-8')

#             # Extract year and filter out 2023 rows
#             df['year'] = df.iloc[:, 0].apply(extract_year)
#             df = df[df['year'] != 2023]
#             df.drop(columns=['year'], inplace=True, errors='ignore')

#             # Save to cleaned2 folder
#             df.to_csv(output_path, index=False)
#             print(f"[✓] 2023 removed → {output_path}")

#         except Exception as e:
#             print(f"[✗] Error processing {file}: {e}")



In [6]:
# import pandas as pd
# import os
# import shutil

# input_folder = 'cleaned2'
# output_hsort = 'HSort2'       # Already created
# output_hunsorted = 'HUnsorted2'  # Already created

# # Helper: Extract (year, month, day) from date string
# def parse_date_info(date_str):
#     date_str = str(date_str).strip()
#     parts = date_str.split(' ')[0].split('-')

#     if len(parts) != 3:
#         return None, None, None

#     if len(parts[0]) == 4:
#         year, month, day = parts
#     else:
#         day, month, year = parts

#     return int(year), int(month), int(day)

# # Track problems
# hsort2_files = []
# hunsorted2_files = []

# # Process all files in cleaned2
# for file in os.listdir(input_folder):
#     if not file.endswith('.csv'):
#         continue

#     path = os.path.join(input_folder, file)
#     try:
#         df = pd.read_csv(path)
#         start_y, start_m, start_d = parse_date_info(df.iloc[0, 0])
#         end_y, end_m, end_d = parse_date_info(df.iloc[-1, 0])
#         years_present = df.iloc[:, 0].apply(lambda d: parse_date_info(d)[0])
#         unique_years = set(years_present.dropna())

#         move_to_hsort = False
#         move_to_hunsorted = False

#         # Check for incomplete start in 2021 or 2022
#         if start_y in [2021, 2022] and (start_m != 1 or start_d != 1):
#             move_to_hsort = True

#         # Check for unwanted years (pre-2021 or 2023)
#         if 2023 in unique_years or any(y < 2021 for y in unique_years):
#             move_to_hunsorted = True

#         # Copy to appropriate folders
#         if move_to_hsort:
#             shutil.copy(path, os.path.join(output_hsort, file))
#             hsort2_files.append(file)

#         if move_to_hunsorted:
#             shutil.copy(path, os.path.join(output_hunsorted, file))
#             hunsorted2_files.append(file)

#     except Exception as e:
#         print(f"❗ Error processing {file}: {e}")

# # Final report
# print(f"\n✅ Validation complete.")
# print(f"🔶 Incomplete 2021/2022 starts → {len(hsort2_files)} files moved to HSort2/")
# print(f"🔴 Unwanted years (pre-2021 or 2023) → {len(hunsorted2_files)} files moved to HUnsorted2/")

# if hsort2_files:
#     print("\n🔶 Files in HSort2:")
#     for f in hsort2_files:
#         print(f"   - {f}")

# if hunsorted2_files:
#     print("\n🔴 Files in HUnsorted2:")
#     for f in hunsorted2_files:
#         print(f"   - {f}")


In [7]:
# import pandas as pd
# import os

# # Folder with cleaned data
# folder_path = 'cleaned2'

# # Pick first CSV file to inspect
# sample_file = next((f for f in os.listdir(folder_path) if f.endswith('.csv')), None)

# if sample_file:
#     sample_path = os.path.join(folder_path, sample_file)
#     df = pd.read_csv(sample_path)

#     print(f"📁 File: {sample_file}")
#     print("\n📌 Column Names:")
#     for i, col in enumerate(df.columns):
#         print(f"{i:>2}: {col}")

#     print("\n🔍 Data Types:")
#     print(df.dtypes)

#     print("\n🔎 Sample Data (first 3 rows):")
#     display(df.head(3))
# else:
#     print("No CSV files found in cleaned2.")

    

In [8]:
# import pandas as pd
# import os

# folder_path = 'cleaned3'
# unique_columns = []

# # Scan each file and collect unseen columns
# for file in os.listdir(folder_path):
#     if file.endswith('.csv'):
#         file_path = os.path.join(folder_path, file)
#         try:
#             df = pd.read_csv(file_path)
#             for col in df.columns:
#                 if col not in unique_columns:
#                     unique_columns.append(col)
#         except Exception as e:
#             print(f"⚠️ Error reading {file}: {e}")

# # Show only the final list of unique columns
# print("📌 Unique Columns Across All Files:\n")
# for col in unique_columns:
#     print(f"- {col}")
# print(f"\n🔢 Total unique columns found: {len(unique_columns)}")


In [9]:
# import pandas as pd
# import os

# folder_path = 'cleaned3'

# required_columns = [
#     "From Date", "To Date", "PM2.5 (ug/m3)", "PM10 (ug/m3)", "NO (ug/m3)", "NO2 (ug/m3)", 
#     "NOx (ppb)", "SO2 (ug/m3)", "CO (mg/m3)", "Ozone (ug/m3)", "NH3 (ug/m3)", 
#     "Benzene (ug/m3)", "Toluene (ug/m3)", "Xylene (ug/m3)", "Temp (degree C)", 
#     "RH (%)", "WS (m/s)", "WD (deg)", "SR (W/mt2)", "BP (mmHg)", "RF (mm)"
# ]

# compatible_count = 0
# total_files = 0

# for file in os.listdir(folder_path):
#     if file.endswith('.csv'):
#         total_files += 1
#         file_path = os.path.join(folder_path, file)
#         try:
#             df = pd.read_csv(file_path)
#             if all(col in df.columns for col in required_columns):
#                 compatible_count += 1
#         except Exception as e:
#             print(f"⚠️ Error in {file}: {e}")

# print(f"\n✅ Files with all required columns: {compatible_count} / {total_files}")


In [10]:
# import pandas as pd
# import os

# # Define folder and required columns
# folder_path = 'cleaned3'
# required_columns = [
#     "From Date", "To Date", "PM2.5 (ug/m3)", "PM10 (ug/m3)", "NO (ug/m3)", "NO2 (ug/m3)", 
#     "NOx (ppb)", "SO2 (ug/m3)", "CO (mg/m3)", "Ozone (ug/m3)", "NH3 (ug/m3)", 
#     "Benzene (ug/m3)", "Toluene (ug/m3)", "Xylene (ug/m3)", "Temp (degree C)", 
#     "RH (%)", "WS (m/s)", "WD (deg)", "SR (W/mt2)", "BP (mmHg)", "RF (mm)"
# ]

# # Minimum required columns for 85%+ coverage
# min_required = 18
# files_with_85_percent = []

# total_files = 0

# for file in os.listdir(folder_path):
#     if file.endswith('.csv'):
#         total_files += 1
#         file_path = os.path.join(folder_path, file)
#         try:
#             df = pd.read_csv(file_path)
#             present_cols = sum(col in df.columns for col in required_columns)
#             if present_cols >= min_required:
#                 files_with_85_percent.append(file)
#         except Exception as e:
#             print(f"⚠️ Error reading {file}: {e}")

# # Output the result
# print(f"\n✅ Files with ≥85% required columns (≥18/21): {len(files_with_85_percent)}")
# print(f"📊 Total files scanned: {total_files}")



In [11]:
# import pandas as pd
# import os

# folder_path = 'cleaned3'
# required_columns = set([
#     "From Date", "To Date", "PM2.5 (ug/m3)", "PM10 (ug/m3)", "NO (ug/m3)",
#     "NO2 (ug/m3)", "NOx (ppb)", "SO2 (ug/m3)", "CO (mg/m3)", "Ozone (ug/m3)",
#     "Temp (degree C)", "RH (%)", "WS (m/s)", "WD (deg)", "SR (W/mt2)",
#     "BP (mmHg)", "RF (mm)", "NH3 (ug/m3)"
# ])

# valid_files = []

# for file in os.listdir(folder_path):
#     if file.endswith('.csv'):
#         try:
#             df = pd.read_csv(os.path.join(folder_path, file))
#             file_cols = set(df.columns)
#             if required_columns.issubset(file_cols):
#                 valid_files.append(file)
#         except Exception as e:
#             print(f"⚠️ Error reading {file}: {e}")

# print(f"\n📂 Files with ALL required 18 columns (possibly more too): {len(valid_files)}")


In [12]:
# import pandas as pd
# import os

# folder = "cleaned2"
# exact_match = "2021-01-01 00:00:00"
# alt_match = "01-01-2021 00:00:00"

# count_exact = 0
# count_2021_not_exact = 0
# count_other = 0

# for file in os.listdir(folder):
#     if not file.endswith('.csv'):
#         continue
#     path = os.path.join(folder, file)
#     try:
#         df = pd.read_csv(path, nrows=1)
#         start_date = str(df.iloc[0, 0]).strip()
        
#         if start_date == exact_match or start_date == alt_match:
#             count_exact += 1
#         elif "2021" in start_date:
#             count_2021_not_exact += 1
#         else:
#             count_other += 1
#     except Exception as e:
#         print(f"⚠️ Error reading {file}: {e}")

# print("\n📊 Start Date Summary:")
# print(f"✅ Exact start at 2021-01-01 00:00:00     : {count_exact}")
# print(f"🟡 Starts in 2021 but not exact          : {count_2021_not_exact}")
# print(f"🔴 Starts in 2022 or later (or error)    : {count_other}")


In [13]:
# import pandas as pd
# import os
# from datetime import datetime

# # Folder paths
# input_folder = "cleaned2"
# output_folder = "cleaned3"

# # Create a helper function to parse date from different formats
# def parse_date_safe(date_str):
#     for fmt in ("%Y-%m-%d %H:%M:%S", "%d-%m-%Y %H:%M:%S"):
#         try:
#             return datetime.strptime(date_str.strip(), fmt).date()
#         except:
#             continue
#     return None

# accepted_files = []
# skipped_files = []

# for file in os.listdir(input_folder):
#     if not file.endswith(".csv"):
#         continue
#     file_path = os.path.join(input_folder, file)
    
#     try:
#         df = pd.read_csv(file_path, nrows=1)
#         raw_date = str(df.iloc[0, 0])
#         parsed_date = parse_date_safe(raw_date)

#         if parsed_date and datetime(2021, 1, 1).date() <= parsed_date <= datetime(2021, 1, 7).date():
#             # ✅ Keep file
#             accepted_files.append(file)
#             output_path = os.path.join(output_folder, file)
#             pd.read_csv(file_path).to_csv(output_path, index=False)
#         else:
#             skipped_files.append((file, parsed_date))
#     except Exception as e:
#         skipped_files.append((file, f"⚠️ Error: {e}"))

# # 🧾 Summary
# print("📂 Selected files copied to cleaned3/:")
# print(f"✅ Total accepted: {len(accepted_files)}")
# print(f"❌ Skipped or invalid: {len(skipped_files)}")


In [14]:
# import pandas as pd
# import os

# folder = 'cleaned3'
# required_columns = [
#     'From Date',
#     'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)',
#     'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)',
#     'Temp (degree C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)',
#     'BP (mmHg)', 'RF (mm)', 'AT (degree C)'
# ]

# matched_files = 0
# total_files = 0

# for file in os.listdir(folder):
#     if file.endswith('.csv'):
#         total_files += 1
#         path = os.path.join(folder, file)
#         try:
#             df = pd.read_csv(path, nrows=1)
#             if all(col in df.columns for col in required_columns):
#                 matched_files += 1
#         except Exception as e:
#             print(f"⚠️ {file}: {e}")

# print(f"\n📊 Files with ALL required columns: {matched_files} / {total_files}")



In [15]:
# import pandas as pd
# import os

# folder = 'cleaned3'
# required_columns = [
#     'From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)',
#     'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'Temp (degree C)',
#     'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'BP (mmHg)', 'RF (mm)', 'AT (degree C)'
# ]

# missing_count_dist = {}
# files_with_missing = []

# for file in os.listdir(folder):
#     if file.endswith('.csv'):
#         path = os.path.join(folder, file)
#         try:
#             df = pd.read_csv(path, nrows=1)
#             present_cols = df.columns
#             missing_cols = [col for col in required_columns if col not in present_cols]
#             missing_count = len(missing_cols)

#             # Track frequency
#             if missing_count in missing_count_dist:
#                 missing_count_dist[missing_count] += 1
#             else:
#                 missing_count_dist[missing_count] = 1

#             # Track specific files with >3 missing cols
#             if missing_count > 3:
#                 files_with_missing.append((file, missing_cols))

#         except Exception as e:
#             print(f"⚠️ Error reading {file}: {e}")

# # Summary
# print("📊 Distribution of Missing Columns per File (out of 18 required):\n")
# for count in sorted(missing_count_dist):
#     print(f"🧩 {count} missing: {missing_count_dist[count]} files")

# if files_with_missing:
#     print("\n❗ Files with >3 missing columns:")
#     for f, missing in files_with_missing:
#         print(f"  - {f} ➤ Missing: {missing}")


In [16]:
# import os
# import pandas as pd
# import shutil

# # Define folder paths
# source_folder = 'cleaned3'
# target_folder = 'cleaned4'  # Make sure this exists

# # Define the 18 required columns
# required_columns = [
#     'From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)',
#     'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'Temp (degree C)',
#     'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'BP (mmHg)', 'RF (mm)', 'AT (degree C)'
# ]

# # Move qualifying files (0–3 missing columns)
# moved_files = []

# for file in os.listdir(source_folder):
#     if file.endswith('.csv'):
#         file_path = os.path.join(source_folder, file)
#         try:
#             df = pd.read_csv(file_path, nrows=1)
#             present_cols = df.columns
#             missing = [col for col in required_columns if col not in present_cols]
#             if len(missing) <= 3:
#                 shutil.copy(file_path, os.path.join(target_folder, file))
#                 moved_files.append(file)
#         except Exception as e:
#             print(f"⚠️ Error reading {file}: {e}")

# print(f"✅ {len(moved_files)} files moved to '{target_folder}' for modeling-ready preprocessing.")


In [17]:
# import os

# folder = 'cleaned4'  # Make sure this matches your actual folder name

# # Rename files from _C3.csv to _C4.csv
# renamed_files = []

# for filename in os.listdir(folder):
#     if filename.endswith('_C3.csv'):
#         old_path = os.path.join(folder, filename)
#         new_filename = filename.replace('_C3.csv', '_C4.csv')
#         new_path = os.path.join(folder, new_filename)
#         os.rename(old_path, new_path)
#         renamed_files.append(new_filename)

# print(f"✅ Renamed {len(renamed_files)} files to _C4.csv format.")


In [18]:
# import os
# import pandas as pd

# input_folder = 'cleaned4'
# output_folder = 'cleaned5'
# os.makedirs(output_folder, exist_ok=True)

# # ✅ Define the 18 required columns
# required_columns = [
#     'From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)',
#     'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'Temp (degree C)',
#     'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'BP (mmHg)', 'RF (mm)', 'AT (degree C)'
# ]

# processed_files = []

# for filename in os.listdir(input_folder):
#     if filename.endswith('_C4.csv'):
#         input_path = os.path.join(input_folder, filename)
#         try:
#             df = pd.read_csv(input_path)

#             # 🔄 Keep only required columns, add any missing ones with NaN
#             df = df.reindex(columns=required_columns)

#             # 📄 Save to cleaned5/ with _C5.csv suffix
#             new_filename = filename.replace('_C4.csv', '_C5.csv')
#             output_path = os.path.join(output_folder, new_filename)
#             df.to_csv(output_path, index=False)
#             processed_files.append(new_filename)

#         except Exception as e:
#             print(f"⚠️ Error processing {filename}: {e}")

# print(f"✅ Processed and standardized {len(processed_files)} files to '{output_folder}/'")


In [19]:
# import os
# import pandas as pd

# folder = 'cleaned4'
# expected_rows = 17520
# mismatched_files = []

# for file in os.listdir(folder):
#     if file.endswith('.csv'):
#         path = os.path.join(folder, file)
#         try:
#             df = pd.read_csv(path)
#             row_count = df.shape[0]
#             if row_count != expected_rows:
#                 mismatched_files.append((file, row_count))
#         except Exception as e:
#             print(f"⚠️ Error reading {file}: {e}")

# print(f"\n🔍 Files with row count NOT equal to {expected_rows}:\n")
# for fname, rows in mismatched_files:
#     print(f"{fname}: {rows} rows")

# print(f"\n✅ Total files scanned: {len(os.listdir(folder))}")
# print(f"❌ Files with incomplete rows: {len(mismatched_files)}")



In [20]:
# import os
# import shutil

# # File identifiers (without the _C5.csv suffix)
# incomplete_basenames = ['MH013', 'TG002', 'TN011']

# # Define folders
# source_folder = 'cleaned5'
# trash_folder = 'trash101'

# # Make sure trash folder exists
# os.makedirs(trash_folder, exist_ok=True)

# removed_count = 0

# for base in incomplete_basenames:
#     filename = f"{base}_C5.csv"
#     source_path = os.path.join(source_folder, filename)
#     trash_path = os.path.join(trash_folder, filename)

#     if os.path.exists(source_path):
#         shutil.move(source_path, trash_path)
#         print(f"🗑️ Moved to trash101/: {filename}")
#         removed_count += 1
#     else:
#         print(f"❌ Not found in cleaned5/: {filename}")

# # Final count of remaining CSVs in cleaned5
# remaining_files = [f for f in os.listdir(source_folder) if f.endswith('.csv')]

# print(f"\n📦 Files moved to trash101: {removed_count}")
# print(f"📂 Remaining CSVs in cleaned5/: {len(remaining_files)}")


In [21]:
# import os
# import pandas as pd

# folder_path = 'cleaned5'

# # ✅ Final required columns (already dropped Benzene, Toluene, etc.)
# required_columns = [
#     'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)',
#     'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)',
#     'Temp (degree C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 
#     'SR (W/mt2)', 'AT (degree C)'
# ]

# allowed_missing = 3
# acceptable_files = []
# fully_matching_files = []
# file_missing_counts = {}

# for file in os.listdir(folder_path):
#     if file.endswith('.csv'):
#         path = os.path.join(folder_path, file)
#         try:
#             df = pd.read_csv(path)
#             columns_present = df.columns.tolist()
#             missing = [col for col in required_columns if col not in columns_present]
#             missing_count = len(missing)
#             file_missing_counts[file] = missing_count
#             if missing_count == 0:
#                 fully_matching_files.append(file)
#             if missing_count <= allowed_missing:
#                 acceptable_files.append(file)
#         except Exception as e:
#             print(f"⚠️ Error reading {file}: {e}")

# print("📊 Relaxed Column Match Summary:")
# print(f"✅ Fully matched (0 missing)         : {len(fully_matching_files)}")
# print(f"🟢 Accepted (≤ 3 missing columns)     : {len(acceptable_files)}")
# print(f"❌ Not accepted (> 3 missing columns) : {len(file_missing_counts) - len(acceptable_files)}")
# print(f"📦 Total files scanned                : {len(file_missing_counts)}")


In [22]:
# import pandas as pd
# import os

# input_dir = "cleaned5"
# output_dir = "cleaned6"
# os.makedirs(output_dir, exist_ok=True)

# required_columns = [
#     'From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)',
#     'NOx (ppb)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)',
#     'Temp (degree C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'BP (mmHg)'
# ]

# # Step 1: Compute global column-wise means (excluding 'From Date')
# global_means = {col: [] for col in required_columns if col != 'From Date'}

# # Accumulate data for global means
# for file in os.listdir(input_dir):
#     if file.endswith('.csv'):
#         df = pd.read_csv(os.path.join(input_dir, file))
#         for col in global_means:
#             if col in df.columns and not df[col].isna().all():
#                 global_means[col].extend(df[col].dropna().tolist())

# # Final mean per column
# column_fill_values = {
#     col: pd.Series(vals).mean() if vals else None
#     for col, vals in global_means.items()
# }

# # Step 2: Fill fully-NaN columns in each file using these means
# for file in os.listdir(input_dir):
#     if file.endswith('.csv'):
#         path = os.path.join(input_dir, file)
#         df = pd.read_csv(path)

#         for col in required_columns:
#             if col != 'From Date' and col in df.columns:
#                 if df[col].isna().all():
#                     fill_value = column_fill_values[col]
#                     if fill_value is not None:
#                         df[col] = fill_value
#                         print(f"🛠️ Filled NaNs in '{col}' for {file} with mean: {round(fill_value, 2)}")

#         df.to_csv(os.path.join(output_dir, file.replace('_C5.csv', '_C6.csv')), index=False)

# print("\n✅ Imputation complete. Files saved in 'cleaned6/' with updated column values.")


In [23]:
# import pandas as pd
# import os

# cleaned5_dir = "cleaned5"
# cleaned6_dir = "cleaned6"

# required_columns = [
#     'From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)',
#     'NOx (ppb)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)',
#     'Temp (degree C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'BP (mmHg)'
# ]

# incorrect_files = {}
# checked_files = 0

# for file in os.listdir(cleaned6_dir):
#     if not file.endswith('.csv'):
#         continue

#     file5_path = os.path.join(cleaned5_dir, file.replace('_C6.csv', '_C5.csv'))
#     file6_path = os.path.join(cleaned6_dir, file)

#     if not os.path.exists(file5_path):
#         continue

#     df5 = pd.read_csv(file5_path)
#     df6 = pd.read_csv(file6_path)

#     wrong_columns = []
#     for col in required_columns:
#         if col not in df5.columns or col not in df6.columns:
#             continue

#         # If column was NOT fully NaN in cleaned5, it should remain unchanged
#         if not df5[col].isna().all():
#             if not df5[col].equals(df6[col]):
#                 wrong_columns.append(col)

#     if wrong_columns:
#         incorrect_files[file] = wrong_columns

#     checked_files += 1

# # 🔍 Summary Report
# print(f"\n📂 Files checked: {checked_files}")
# if incorrect_files:
#     print(f"\n❌ Files with incorrect overwrites: {len(incorrect_files)}\n")
#     for file, cols in incorrect_files.items():
#         print(f"- {file}: Wrongly modified columns ➜ {cols}")
# else:
#     print("\n✅ All columns were filled only where needed. Imputation verified successfully.")


In [25]:
# import os
# import re
# from collections import defaultdict

# def summarize_tif_metadata(folder_path):
#     tif_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.tif')]
    
#     band_counts = defaultdict(int)
#     time_stamps = set()
#     pattern = re.compile(r'_([0-9]{4})_.*?_IMG_([A-Z0-9]+)\.tif$', re.IGNORECASE)

#     for fname in tif_files:
#         match = pattern.search(fname)
#         if match:
#             time_str = match.group(1)
#             band = match.group(2)
#             time_stamps.add(time_str)
#             band_counts[band] += 1
#         else:
#             print(f"⚠️ Unmatched filename format: {fname}")

#     print("📊 Summary:")
#     print(f"Total .tif files: {len(tif_files)}")
#     print(f"Total unique time slots: {len(time_stamps)}")
#     print(f"Time slots (sorted): {sorted(time_stamps)}")
#     print("\n🛰️ Band counts:")
#     for band, count in band_counts.items():
#         print(f"- {band}: {count} files")

# # Example usage
# folder_path = "INSAT_img_folder/HR_2021_TIR1_WV"
# summarize_tif_metadata(folder_path)


📊 Summary:
Total .tif files: 3580
Total unique time slots: 101
Time slots (sorted): ['0758', '0802', '0807', '0815', '0819', '0824', '0828', '0832', '0837', '0845', '0849', '0854', '0855', '0858', '0902', '0907', '0915', '0919', '0924', '0928', '0932', '0937', '0945', '0949', '0954', '0958', '1002', '1007', '1015', '1018', '1019', '1024', '1028', '1032', '1037', '1045', '1049', '1054', '1058', '1102', '1107', '1115', '1119', '1124', '1128', '1132', '1137', '1145', '1149', '1154', '1158', '1202', '1207', '1215', '1219', '1224', '1228', '1232', '1237', '1245', '1249', '1254', '1255', '1258', '1302', '1307', '1315', '1319', '1324', '1328', '1332', '1337', '1345', '1349', '1354', '1358', '1402', '1407', '1415', '1419', '1424', '1428', '1432', '1437', '1445', '1449', '1454', '1458', '1502', '1507', '1515', '1519', '1524', '1528', '1532', '1537', '1545', '1549', '1554', '1558', '1602']

🛰️ Band counts:
- TIR1: 1790 files
- WV: 1790 files


In [26]:
# import os

# def extract_time_and_band(filename):
#     parts = filename.split('_')
#     time = parts[2]  # e.g., '0815'
#     band = parts[-1].replace('.tif', '')  # e.g., 'TIR1' or 'WV'
#     return time, band

# def check_band_alternation_order(folder_path):
#     tif_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.tif')])

#     anomalies = []
#     for i in range(0, len(tif_files) - 1, 2):  # Check in pairs
#         file1, file2 = tif_files[i], tif_files[i + 1]
#         time1, band1 = extract_time_and_band(file1)
#         time2, band2 = extract_time_and_band(file2)

#         if time1 != time2:
#             anomalies.append((file1, file2, '❌ Time mismatch'))
#         elif band1 == band2:
#             anomalies.append((file1, file2, '❌ Same band repeated'))
#         elif {band1, band2} != {'TIR1', 'WV'}:
#             anomalies.append((file1, file2, '❌ Unexpected band pair'))

#     return anomalies

# # Set full folder path to Documents (Windows)
# folder = r"INSAT_img_folder/HR_2021_TIR1_WV"  # ← Replace with your actual Windows username
# issues = check_band_alternation_order(folder)

# if not issues:
#     print("✅ All files alternate correctly with matching timestamps and expected bands (TIR1/WV).")
# else:
#     print(f"⚠️ Found {len(issues)} anomalies:")
#     for f1, f2, reason in issues:
#         print(f"  - {f1} | {f2} → {reason}")



✅ All files alternate correctly with matching timestamps and expected bands (TIR1/WV).


In [3]:
# import os
# import re
# from datetime import datetime, timedelta, time

# # Set folder path
# folder_path = "INSAT_img_folder/HR_2021_TIR1_WV"

# # Define valid slots from 08:00 to 16:00 in 30-min steps
# slot_times = [time(hour=h, minute=m) for h in range(8, 17) for m in (0, 30) if not (h == 16 and m == 30)]

# # Roundability threshold
# tolerance_minutes = 10

# # Categorization buckets
# roundable_files = []
# not_roundable = []
# outside_valid_window = []

# # Check closeness to any slot
# def is_within_window(t):
#     return slot_times[0] <= t <= slot_times[-1]

# def is_roundable(t):
#     for slot in slot_times:
#         slot_dt = datetime.combine(datetime.today(), slot)
#         t_dt = datetime.combine(datetime.today(), t)
#         delta = abs((slot_dt - t_dt).total_seconds()) / 60
#         if delta <= tolerance_minutes:
#             return True
#     return False

# # Loop through files
# all_files = os.listdir(folder_path)
# for file in all_files:
#     match = re.search(r"_(\d{4})_L1B_STD", file)
#     if not match:
#         continue

#     try:
#         time_str = match.group(1)
#         file_time = datetime.strptime(time_str, "%H%M").time()
#     except:
#         continue

#     if not is_within_window(file_time):
#         outside_valid_window.append(file)
#     elif is_roundable(file_time):
#         roundable_files.append(file)
#     else:
#         not_roundable.append(file)

# # Output
# print("📊 File Categorization Summary:")
# print(f"Total files scanned:       {len(all_files)}")
# print(f"✔ Within valid window:     {len(roundable_files) + len(not_roundable)} file(s)")
# print(f"↪ Roundable to slot:       {len(roundable_files)} file(s)")
# print(f"⚠ Not roundable:           {len(not_roundable)} file(s)")
# print(f"❌ Outside valid window:    {len(outside_valid_window)} file(s)")

# # Debug print for first few not roundable files
# print("\n🔍 Sample Not-Roundable Files (first 10):")
# for f in not_roundable[:10]:
#     ts = re.search(r"_(\d{4})_L1B_STD", f)
#     if ts:
#         t = datetime.strptime(ts.group(1), "%H%M").time()
#         print(f"{f} → {t}")



📊 File Categorization Summary:
Total files scanned:       3580
✔ Within valid window:     3558 file(s)
↪ Roundable to slot:       576 file(s)
⚠ Not roundable:           2982 file(s)
❌ Outside valid window:    22 file(s)

🔍 Sample Not-Roundable Files (first 10):
3RIMG_01NOV2021_0815_L1B_STD_V01R00_IMG_TIR1.tif → 08:15:00
3RIMG_01NOV2021_0815_L1B_STD_V01R00_IMG_WV.tif → 08:15:00
3RIMG_01NOV2021_0845_L1B_STD_V01R00_IMG_TIR1.tif → 08:45:00
3RIMG_01NOV2021_0845_L1B_STD_V01R00_IMG_WV.tif → 08:45:00
3RIMG_01NOV2021_0915_L1B_STD_V01R00_IMG_TIR1.tif → 09:15:00
3RIMG_01NOV2021_0915_L1B_STD_V01R00_IMG_WV.tif → 09:15:00
3RIMG_01NOV2021_0945_L1B_STD_V01R00_IMG_TIR1.tif → 09:45:00
3RIMG_01NOV2021_0945_L1B_STD_V01R00_IMG_WV.tif → 09:45:00
3RIMG_01NOV2021_1015_L1B_STD_V01R00_IMG_TIR1.tif → 10:15:00
3RIMG_01NOV2021_1015_L1B_STD_V01R00_IMG_WV.tif → 10:15:00


In [4]:
# from pathlib import Path
# from collections import defaultdict
# from datetime import datetime

# # Root folder containing all files
# folder = Path("INSAT_img_folder/HR_2021_TIR1_WV")

# # Initialize structure to hold counts
# datewise_counts = defaultdict(lambda: {'valid': 0, 'invalid': 0})

# # Extract date and minute from each filename
# for file in folder.glob("*.tif"):
#     name = file.name
#     try:
#         # Format: 3RIMG_01NOV2021_0815_...
#         parts = name.split("_")
#         date_str = parts[1]  # e.g., 01NOV2021
#         time_str = parts[2]  # e.g., 0815

#         # Convert to datetime
#         dt = datetime.strptime(date_str + time_str, "%d%b%Y%H%M")

#         # Keyed by date in YYYY-MM-DD format
#         date_key = dt.strftime("%Y-%m-%d")

#         if dt.minute in {0, 15, 30, 45}:
#             datewise_counts[date_key]['valid'] += 1
#         else:
#             datewise_counts[date_key]['invalid'] += 1
#     except Exception as e:
#         print(f"⚠ Failed to parse: {name}")

# # Show summary
# print("📅 Per-Day Validity Summary:\n")
# for date in sorted(datewise_counts):
#     counts = datewise_counts[date]
#     print(f"{date}: ✅ Valid = {counts['valid']:>3}, ❌ Invalid = {counts['invalid']:>3}")


📅 Per-Day Validity Summary:

2021-09-01: ✅ Valid =  32, ❌ Invalid =   0
2021-09-02: ✅ Valid =  32, ❌ Invalid =   0
2021-09-03: ✅ Valid =  32, ❌ Invalid =   0
2021-09-04: ✅ Valid =  32, ❌ Invalid =   0
2021-09-05: ✅ Valid =  32, ❌ Invalid =   0
2021-09-06: ✅ Valid =  32, ❌ Invalid =   0
2021-09-07: ✅ Valid =  32, ❌ Invalid =   0
2021-09-08: ✅ Valid =  32, ❌ Invalid =   0
2021-09-09: ✅ Valid =  32, ❌ Invalid =   0
2021-09-10: ✅ Valid =  32, ❌ Invalid =   0
2021-09-11: ✅ Valid =  32, ❌ Invalid =   0
2021-09-12: ✅ Valid =  32, ❌ Invalid =   0
2021-09-13: ✅ Valid =  32, ❌ Invalid =   0
2021-09-14: ✅ Valid =  32, ❌ Invalid =   0
2021-09-15: ✅ Valid =  32, ❌ Invalid =   0
2021-09-16: ✅ Valid =  32, ❌ Invalid =   0
2021-09-17: ✅ Valid =  32, ❌ Invalid =   2
2021-09-18: ✅ Valid =  32, ❌ Invalid =   0
2021-09-19: ✅ Valid =  32, ❌ Invalid =   0
2021-09-20: ✅ Valid =  32, ❌ Invalid =   0
2021-09-21: ✅ Valid =  30, ❌ Invalid =   0
2021-09-22: ✅ Valid =  32, ❌ Invalid =   0
2021-09-23: ✅ Valid =  32

In [5]:
# from pathlib import Path
# from datetime import datetime
# import re

# # Set your base folder path here
# base_path = Path("INSAT_img_folder/HR_2021_TIR1_WV")  # 🔁 Change this to your folder

# # Regex to extract date and time from filename
# filename_pattern = re.compile(r"_(\d{2}[A-Z]{3}\d{4})_(\d{4})_")

# # Store all invalid times
# invalid_times = []

# # Loop over all .tif files in all subfolders
# for file_path in base_path.rglob("*.tif"):
#     match = filename_pattern.search(file_path.name)
#     if match:
#         date_str, time_str = match.groups()
#         try:
#             dt = datetime.strptime(date_str + time_str, "%d%b%Y%H%M")
#             if dt.minute not in {0, 15, 30, 45}:
#                 invalid_times.append(dt.strftime("%H:%M:%S"))
#         except ValueError:
#             continue

# # Print all invalid times (can be long)
# print("🕒 Invalid times (minutes not 00/15/30/45):")
# for time_str in invalid_times:
#     print(time_str)

# print(f"\n🔢 Total invalid timestamps: {len(invalid_times)}")


🕒 Invalid times (minutes not 00/15/30/45):
07:58:00
07:58:00
08:02:00
08:02:00
08:07:00
08:07:00
08:19:00
08:19:00
08:24:00
08:24:00
08:28:00
08:28:00
08:32:00
08:32:00
08:37:00
08:37:00
08:49:00
08:49:00
08:54:00
08:54:00
08:58:00
08:58:00
09:02:00
09:02:00
09:07:00
09:07:00
09:19:00
09:19:00
09:24:00
09:24:00
09:28:00
09:28:00
09:32:00
09:32:00
09:37:00
09:37:00
09:49:00
09:49:00
09:54:00
09:54:00
09:58:00
09:58:00
10:02:00
10:02:00
10:07:00
10:07:00
10:19:00
10:19:00
10:24:00
10:24:00
10:28:00
10:28:00
10:32:00
10:32:00
10:37:00
10:37:00
10:49:00
10:49:00
10:54:00
10:54:00
10:58:00
10:58:00
11:02:00
11:02:00
11:07:00
11:07:00
11:19:00
11:19:00
11:24:00
11:24:00
11:28:00
11:28:00
11:32:00
11:32:00
11:37:00
11:37:00
11:49:00
11:49:00
11:54:00
11:54:00
11:58:00
11:58:00
12:02:00
12:02:00
12:07:00
12:07:00
12:19:00
12:19:00
12:24:00
12:24:00
12:28:00
12:28:00
12:32:00
12:32:00
12:37:00
12:37:00
12:49:00
12:49:00
12:54:00
12:54:00
12:58:00
12:58:00
13:02:00
13:02:00
13:07:00
13:07:00
13:

In [7]:
# from pathlib import Path
# from datetime import datetime
# import re
# from collections import defaultdict

# # 📁 Folder containing .tif files
# folder_path = Path("INSAT_img_folder/HR_2021_TIR1_WV")

# # ⏱ Valid minutes to check
# valid_minutes = [0, 15, 30, 45]
# minute_count = defaultdict(int)

# # 🔍 Regex to extract date and time from filenames
# filename_pattern = re.compile(r"_(\d{2}[A-Z]{3}\d{4})_(\d{4})_")

# # 🔁 Scan all .tif files in folder
# for file_path in folder_path.rglob("*.tif"):
#     match = filename_pattern.search(file_path.name)
#     if match:
#         date_str, time_str = match.groups()
#         try:
#             dt = datetime.strptime(date_str + time_str, "%d%b%Y%H%M")
#             if dt.minute in valid_minutes:
#                 minute_count[dt.minute] += 1
#         except ValueError:
#             continue

# # 📊 Print grouped result
# print("✔ File Count by Valid Minute:")
# for minute in sorted(valid_minutes):
#     print(f"  - Minute {minute:02d}: {minute_count[minute]} file(s)")


✔ File Count by Valid Minute:
  - Minute 00: 0 file(s)
  - Minute 15: 1418 file(s)
  - Minute 30: 0 file(s)
  - Minute 45: 1422 file(s)


In [2]:
# import os
# from collections import defaultdict

# # Path to your folder
# folder_path = "cleaned6"

# # Dictionary to store states and their unique stations
# state_station_map = defaultdict(set)

# # Iterate over all files in the folder
# for filename in os.listdir(folder_path):
#     if filename.endswith(".csv") and len(filename) >= 7:
#         # Extract state code and station code
#         state_code = filename[:2]
#         station_code = filename.split('_')[0]  # e.g., 'AP001'
        
#         # Add station to the corresponding state set
#         state_station_map[state_code].add(station_code)

# # Print counts
# print("Station count per state in 'cleaned6' folder:\n")
# for state, stations in state_station_map.items():
#     print(f"{state}: {len(stations)} station(s)")


Station count per state in 'cleaned6' folder:

AP: 5 station(s)
AS: 2 station(s)
BR: 10 station(s)
CH: 1 station(s)
DL: 36 station(s)
GJ: 4 station(s)
HR: 23 station(s)
JH: 1 station(s)
KA: 12 station(s)
KL: 6 station(s)
MH: 27 station(s)
ML: 1 station(s)
MP: 10 station(s)
MZ: 1 station(s)
NL: 1 station(s)
PB: 3 station(s)
RJ: 10 station(s)
TG: 5 station(s)
TN: 8 station(s)
TR: 1 station(s)
UP: 24 station(s)
WB: 11 station(s)


In [8]:
# import os
# import shutil

# # Paths
# original_folder = 'INSAT_img_folder/HR_2021_TIR1_WV'
# clean_folder = 'INSAT_img_folder/INSATClean1'
# trounds_folder = 'INSAT_img_folder/TRounds'
# tquarters_folder = 'INSAT_img_folder/TQuarters'

# # Create necessary folders
# os.makedirs(clean_folder, exist_ok=True)
# os.makedirs(trounds_folder, exist_ok=True)
# os.makedirs(tquarters_folder, exist_ok=True)

# # Step 1: Copy all images to clean folder
# for file in os.listdir(original_folder):
#     if file.endswith('.tif'):
#         shutil.copy(os.path.join(original_folder, file), os.path.join(clean_folder, file))

# # Step 2: Process irregular images from INSATClean1
# def get_minute_from_filename(filename):
#     try:
#         parts = filename.split('_')
#         for part in parts:
#             if len(part) == 4 and part.isdigit():
#                 return int(part[2:4])  # Get MM from HHMM
#     except:
#         return None

# def classify_minute(minute):
#     diffs = {
#         '00': abs(minute - 0),
#         '15': abs(minute - 15),
#         '30': abs(minute - 30),
#         '45': abs(minute - 45)
#     }
#     for key in diffs:
#         if diffs[key] > 30:
#             diffs[key] = 60 - diffs[key]
#     closest = min(diffs, key=diffs.get)
#     if diffs[closest] <= 7:
#         return 'TRounds' if closest in ['00', '30'] else 'TQuarters'
#     return None

# # Only move irregular ones from clean_folder
# for filename in os.listdir(clean_folder):
#     if not filename.lower().endswith('.tif'):
#         continue

#     minute = get_minute_from_filename(filename)
#     if minute is None or minute in [0, 15, 30, 45]:
#         continue  # skip regular timestamps

#     category = classify_minute(minute)
#     if category == 'TRounds':
#         shutil.move(os.path.join(clean_folder, filename), os.path.join(trounds_folder, filename))
#     elif category == 'TQuarters':
#         shutil.move(os.path.join(clean_folder, filename), os.path.join(tquarters_folder, filename))


In [10]:
import os
import re
from datetime import datetime, timedelta

# --- CONFIGURATION ---
folder_path = 'INSAT_img_folder/INSATClean1'  # Update this if needed

# --- Build expected timestamp set ---
start_date = datetime(2021, 9, 1)
end_date = datetime(2021, 11, 30)
expected_timestamps = set()

current_date = start_date
while current_date <= end_date:
    for hour in range(8, 16):  # 08 to 15 inclusive
        for minute in [15, 45]:
            ts = current_date.replace(hour=hour, minute=minute, second=0)
            expected_timestamps.add(ts.strftime("%d%b%Y_%H%M").upper())
    current_date += timedelta(days=1)

# --- Extract timestamps from filenames in folder ---
timestamp_pattern = re.compile(r"RIMG_(\d{2}[A-Z]{3}\d{4}_\d{4})_L1B")
found_timestamps = set()

for filename in os.listdir(folder_path):
    match = timestamp_pattern.search(filename)
    if match:
        found_timestamps.add(match.group(1))

# --- Check for missing timestamps ---
missing = sorted(expected_timestamps - found_timestamps)
extra = sorted(found_timestamps - expected_timestamps)

print(f"Total expected timestamps: {len(expected_timestamps)}")
print(f"Timestamps found: {len(found_timestamps)}")
print(f"Missing timestamps: {len(missing)}")
print(f"Extra/unexpected timestamps: {len(extra)}")

if missing:
    print("\nMissing timestamps:")
    for ts in missing:
        print(ts)

if extra:
    print("\nUnexpected timestamps:")
    for ts in extra:
        print(ts)


Total expected timestamps: 1456
Timestamps found: 1420
Missing timestamps: 36
Extra/unexpected timestamps: 0

Missing timestamps:
04OCT2021_0815
04OCT2021_0845
04OCT2021_0915
04OCT2021_0945
04OCT2021_1015
07NOV2021_1115
21SEP2021_0845
25SEP2021_0845
25SEP2021_0915
25SEP2021_0945
25SEP2021_1015
25SEP2021_1045
25SEP2021_1215
25SEP2021_1315
25SEP2021_1415
25SEP2021_1445
26SEP2021_0845
26SEP2021_0915
26SEP2021_0945
26SEP2021_1015
26SEP2021_1045
26SEP2021_1115
26SEP2021_1145
26SEP2021_1215
26SEP2021_1245
26SEP2021_1345
26SEP2021_1415
26SEP2021_1445
26SEP2021_1515
26SEP2021_1545
27SEP2021_1215
28OCT2021_1015
28OCT2021_1045
28OCT2021_1115
30OCT2021_1345
30OCT2021_1415


In [13]:
import os
import re
from datetime import datetime, timedelta

# --- CONFIGURATION ---
clean_folder = 'INSAT_img_folder/INSATClean1'
tquarters_folder = 'INSAT_img_folder/TQuarters'

# --- Generate expected timestamps ---
start_date = datetime(2021, 9, 1)
end_date = datetime(2021, 11, 30)
expected_timestamps = set()

current_date = start_date
while current_date <= end_date:
    for hour in range(8, 16):  # 08 to 15 inclusive
        for minute in [15, 45]:
            ts = current_date.replace(hour=hour, minute=minute, second=0)
            expected_timestamps.add(ts.strftime("%d%b%Y_%H%M").upper())
    current_date += timedelta(days=1)

# --- Extract timestamps from INSATClean1 filenames ---
timestamp_pattern = re.compile(r"RIMG_(\d{2}[A-Z]{3}\d{4}_\d{4})_L1B")
found_timestamps = set()

for filename in os.listdir(clean_folder):
    match = timestamp_pattern.search(filename)
    if match:
        found_timestamps.add(match.group(1))

# --- Find missing timestamps ---
missing_timestamps = sorted(expected_timestamps - found_timestamps)

# --- Scan TQuarters for similar timestamps ---
tquarters_files = os.listdir(tquarters_folder)
tq_timestamp_map = {}  # timestamp_string -> full filename

for fname in tquarters_files:
    match = timestamp_pattern.search(fname)
    if match:
        ts_str = match.group(1)
        tq_timestamp_map[ts_str] = fname

# --- Try to recover missing timestamps using ±5 min tolerance ---
def parse_ts(ts_str):
    return datetime.strptime(ts_str, "%d%b%Y_%H%M")

recovered = {}
unrecoverable = []

for missing_ts_str in missing_timestamps:
    target_dt = parse_ts(missing_ts_str)
    found = False

    for delta_min in range(-5, 6):  # from -5 to +5 inclusive
        check_dt = target_dt + timedelta(minutes=delta_min)
        check_str = check_dt.strftime("%d%b%Y_%H%M").upper()

        if check_str in tq_timestamp_map:
            recovered[missing_ts_str] = tq_timestamp_map[check_str]
            found = True
            break

    if not found:
        unrecoverable.append(missing_ts_str)

# --- Reporting ---
print(f"\nTotal expected timestamps: {len(expected_timestamps)}")
print(f"Found in INSATClean1: {len(found_timestamps)}")
print(f"Missing from INSATClean1: {len(missing_timestamps)}")
print(f"Recovered from TQuarters: {len(recovered)}")
print(f"Still missing after recovery: {len(unrecoverable)}")

if recovered:
    print("\nRecovered mappings (INSATClean1 missing → TQuarters file):")
    for k, v in recovered.items():
        print(f"{k}  ←  {v}")

if unrecoverable:
    print("\nStill unrecoverable timestamps:")
    for ts in unrecoverable:
        print(ts)



Total expected timestamps: 1456
Found in INSATClean1: 1420
Missing from INSATClean1: 36
Recovered from TQuarters: 4
Still missing after recovery: 32

Recovered mappings (INSATClean1 missing → TQuarters file):
25SEP2021_0845  ←  3RIMG_25SEP2021_0849_L1B_STD_V01R00_IMG_WV.tif
25SEP2021_0945  ←  3RIMG_25SEP2021_0949_L1B_STD_V01R00_IMG_WV.tif
26SEP2021_1045  ←  3RIMG_26SEP2021_1049_L1B_STD_V01R00_IMG_WV.tif
26SEP2021_1345  ←  3RIMG_26SEP2021_1349_L1B_STD_V01R00_IMG_WV.tif

Still unrecoverable timestamps:
04OCT2021_0815
04OCT2021_0845
04OCT2021_0915
04OCT2021_0945
04OCT2021_1015
07NOV2021_1115
21SEP2021_0845
25SEP2021_0915
25SEP2021_1015
25SEP2021_1045
25SEP2021_1215
25SEP2021_1315
25SEP2021_1415
25SEP2021_1445
26SEP2021_0845
26SEP2021_0915
26SEP2021_0945
26SEP2021_1015
26SEP2021_1115
26SEP2021_1145
26SEP2021_1215
26SEP2021_1245
26SEP2021_1415
26SEP2021_1445
26SEP2021_1515
26SEP2021_1545
27SEP2021_1215
28OCT2021_1015
28OCT2021_1045
28OCT2021_1115
30OCT2021_1345
30OCT2021_1415


In [14]:
import os
import re
from datetime import datetime

# --- CONFIGURATION ---
tquarters_folder = 'INSAT_img_folder/TQuarters'

# --- Paste the missing timestamps here ---
missing_timestamps_str = """
04OCT2021_0815
04OCT2021_0845
04OCT2021_0915
04OCT2021_0945
04OCT2021_1015
07NOV2021_1115
21SEP2021_0845
25SEP2021_0845
25SEP2021_0915
25SEP2021_0945
25SEP2021_1015
25SEP2021_1045
25SEP2021_1215
25SEP2021_1315
25SEP2021_1415
25SEP2021_1445
26SEP2021_0845
26SEP2021_0915
26SEP2021_0945
26SEP2021_1015
26SEP2021_1045
26SEP2021_1115
26SEP2021_1145
26SEP2021_1215
26SEP2021_1245
26SEP2021_1345
26SEP2021_1415
26SEP2021_1445
26SEP2021_1515
26SEP2021_1545
27SEP2021_1215
28OCT2021_1015
28OCT2021_1045
28OCT2021_1115
30OCT2021_1345
30OCT2021_1415
""".strip().splitlines()

# Convert missing timestamps to datetime objects
def parse_timestamp(ts_str):
    return datetime.strptime(ts_str.strip(), "%d%b%Y_%H%M")

missing_datetimes = [parse_timestamp(ts) for ts in missing_timestamps_str]

# Build dictionary with (date, hour) as key and list of target minutes
from collections import defaultdict
missing_by_hour = defaultdict(list)
for dt in missing_datetimes:
    missing_by_hour[(dt.date(), dt.hour)].append(dt.minute)

# Regex to extract timestamp from TQuarters filenames
pattern = re.compile(r'RIMG_(\d{2}[A-Z]{3}\d{4}_\d{4})_L1B')

# --- Search TQuarters folder ---
candidates_found = []

for fname in os.listdir(tquarters_folder):
    match = pattern.search(fname)
    if not match:
        continue
    ts_str = match.group(1)
    try:
        ts_dt = datetime.strptime(ts_str, "%d%b%Y_%H%M")
    except:
        continue

    key = (ts_dt.date(), ts_dt.hour)
    if key in missing_by_hour:
        for target_minute in missing_by_hour[key]:
            if abs(ts_dt.minute - target_minute) <= 5:
                candidates_found.append((ts_str, fname))
                break  # Only match once per file

# --- Print Results ---
print(f"Total candidate replacements found: {len(candidates_found)}\n")
for ts_str, fname in candidates_found:
    print(f"Possible replacement: {ts_str} -> {fname}")


Total candidate replacements found: 8

Possible replacement: 25SEP2021_0849 -> 3RIMG_25SEP2021_0849_L1B_STD_V01R00_IMG_TIR1.tif
Possible replacement: 25SEP2021_0849 -> 3RIMG_25SEP2021_0849_L1B_STD_V01R00_IMG_WV.tif
Possible replacement: 25SEP2021_0949 -> 3RIMG_25SEP2021_0949_L1B_STD_V01R00_IMG_TIR1.tif
Possible replacement: 25SEP2021_0949 -> 3RIMG_25SEP2021_0949_L1B_STD_V01R00_IMG_WV.tif
Possible replacement: 26SEP2021_1049 -> 3RIMG_26SEP2021_1049_L1B_STD_V01R00_IMG_TIR1.tif
Possible replacement: 26SEP2021_1049 -> 3RIMG_26SEP2021_1049_L1B_STD_V01R00_IMG_WV.tif
Possible replacement: 26SEP2021_1349 -> 3RIMG_26SEP2021_1349_L1B_STD_V01R00_IMG_TIR1.tif
Possible replacement: 26SEP2021_1349 -> 3RIMG_26SEP2021_1349_L1B_STD_V01R00_IMG_WV.tif


In [16]:
import os
import re
from datetime import datetime
from collections import defaultdict

# --- CONFIGURATION ---
tquarters_folder = 'INSAT_img_folder/TQuarters'

# --- Paste the missing timestamps here ---
missing_timestamps_str = """
04OCT2021_0815
04OCT2021_0845
04OCT2021_0915
04OCT2021_0945
04OCT2021_1015
07NOV2021_1115
21SEP2021_0845
25SEP2021_0845
25SEP2021_0915
25SEP2021_0945
25SEP2021_1015
25SEP2021_1045
25SEP2021_1215
25SEP2021_1315
25SEP2021_1415
25SEP2021_1445
26SEP2021_0845
26SEP2021_0915
26SEP2021_0945
26SEP2021_1015
26SEP2021_1045
26SEP2021_1115
26SEP2021_1145
26SEP2021_1215
26SEP2021_1245
26SEP2021_1345
26SEP2021_1415
26SEP2021_1445
26SEP2021_1515
26SEP2021_1545
27SEP2021_1215
28OCT2021_1015
28OCT2021_1045
28OCT2021_1115
30OCT2021_1345
30OCT2021_1415
""".strip().splitlines()

# Convert missing timestamps to datetime objects
def parse_timestamp(ts_str):
    return datetime.strptime(ts_str.strip(), "%d%b%Y_%H%M")

missing_datetimes = [parse_timestamp(ts) for ts in missing_timestamps_str]

# Build dictionary with (date, hour) as key and list of target minutes
missing_by_hour = defaultdict(list)
for dt in missing_datetimes:
    missing_by_hour[(dt.date(), dt.hour)].append(dt.minute)

# Regex to extract timestamp from TQuarters filenames
pattern = re.compile(r'RIMG_(\d{2}[A-Z]{3}\d{4}_\d{4})_L1B')

# --- Search TQuarters folder ---
candidates_found = []

for fname in os.listdir(tquarters_folder):
    match = pattern.search(fname)
    if not match:
        continue
    ts_str = match.group(1)
    try:
        ts_dt = datetime.strptime(ts_str, "%d%b%Y_%H%M")
    except:
        continue

    key = (ts_dt.date(), ts_dt.hour)
    if key in missing_by_hour:
        for target_minute in missing_by_hour[key]:
            if abs(ts_dt.minute - target_minute) <= 7:
                candidates_found.append((ts_str, fname))
                break  # Only match once per file

# --- Print Results ---
print(f"Total candidate replacements found: {len(candidates_found)}\n")
for ts_str, fname in candidates_found:
    print(f"Possible replacement: {ts_str} -> {fname}")


Total candidate replacements found: 8

Possible replacement: 25SEP2021_0849 -> 3RIMG_25SEP2021_0849_L1B_STD_V01R00_IMG_TIR1.tif
Possible replacement: 25SEP2021_0849 -> 3RIMG_25SEP2021_0849_L1B_STD_V01R00_IMG_WV.tif
Possible replacement: 25SEP2021_0949 -> 3RIMG_25SEP2021_0949_L1B_STD_V01R00_IMG_TIR1.tif
Possible replacement: 25SEP2021_0949 -> 3RIMG_25SEP2021_0949_L1B_STD_V01R00_IMG_WV.tif
Possible replacement: 26SEP2021_1049 -> 3RIMG_26SEP2021_1049_L1B_STD_V01R00_IMG_TIR1.tif
Possible replacement: 26SEP2021_1049 -> 3RIMG_26SEP2021_1049_L1B_STD_V01R00_IMG_WV.tif
Possible replacement: 26SEP2021_1349 -> 3RIMG_26SEP2021_1349_L1B_STD_V01R00_IMG_TIR1.tif
Possible replacement: 26SEP2021_1349 -> 3RIMG_26SEP2021_1349_L1B_STD_V01R00_IMG_WV.tif


In [2]:
import os

# Step 1: Your list of missing datetime stamps
missing_datetimes = [
    "04OCT2021_0815", "04OCT2021_0845", "04OCT2021_0915", "04OCT2021_0945", "04OCT2021_1015",
    "07NOV2021_1115", "21SEP2021_0845", "25SEP2021_0845", "25SEP2021_0915", "25SEP2021_0945",
    "25SEP2021_1015", "25SEP2021_1045", "25SEP2021_1215", "25SEP2021_1315", "25SEP2021_1415",
    "25SEP2021_1445", "26SEP2021_0845", "26SEP2021_0915", "26SEP2021_0945", "26SEP2021_1015",
    "26SEP2021_1045", "26SEP2021_1115", "26SEP2021_1145", "26SEP2021_1215", "26SEP2021_1245",
    "26SEP2021_1345", "26SEP2021_1415", "26SEP2021_1445", "26SEP2021_1515", "26SEP2021_1545",
    "27SEP2021_1215", "28OCT2021_1015", "28OCT2021_1045", "28OCT2021_1115", "30OCT2021_1345",
    "30OCT2021_1415"
]

# Step 2: Extract just the dates (before '_')
missing_dates = sorted(set(dt.split('_')[0] for dt in missing_datetimes))

# Step 3: Scan TRounds folder for filenames containing any of the missing dates
trounds_folder = "INSAT_img_folder/TQuarters"
found_dates = set()

for filename in os.listdir(trounds_folder):
    for date in missing_dates:
        if date in filename:
            found_dates.add(date)

# Step 4: Print which dates were found or not found
print("Missing dates NOT present in TQuarters:")
for date in missing_dates:
    if date not in found_dates:
        print(f" - {date}")


Missing dates NOT present in TQuarters:
 - 04OCT2021
 - 07NOV2021
 - 21SEP2021
 - 27SEP2021
 - 28OCT2021
 - 30OCT2021


In [13]:
# import os
# import shutil
# import re

# def step1_copy_and_prepare(raw_folder_path):
#     if not os.path.exists(raw_folder_path):
#         print(f"Raw folder does not exist: {raw_folder_path}")
#         return

#     raw_folder_name = os.path.basename(raw_folder_path)
    
#     # Extract state abbreviation and year from folder name like 'HR_2022_TIR1_WV'
#     match = re.match(r"([A-Z]{2})_(\d{4})", raw_folder_name)
#     if not match:
#         print("Folder name must begin with pattern like 'HR_2022_'")
#         return

#     state_abbr, year = match.group(1), match.group(2)
#     base_dir = os.path.dirname(raw_folder_path)

#     # Define target folders
#     clean_folder = os.path.join(base_dir, f"INSATClean1_{state_abbr}_{year}")
#     tquarters_folder = os.path.join(base_dir, f"TQuarters_{year}")
#     trounds_folder = os.path.join(base_dir, f"TRounds_{year}")
#     out_of_range_folder = os.path.join(base_dir, f"TOutofRange_{year}")

#     # Create folders if they don't exist
#     for folder in [clean_folder, tquarters_folder, trounds_folder, out_of_range_folder]:
#         os.makedirs(folder, exist_ok=True)

#     # Copy .tif files to clean folder
#     tif_files = [f for f in os.listdir(raw_folder_path) if f.lower().endswith(".tif")]
#     for file in tif_files:
#         src = os.path.join(raw_folder_path, file)
#         dst = os.path.join(clean_folder, file)
#         shutil.copy2(src, dst)

#     print(f"Step 1: Copied {len(tif_files)} .tif files to {clean_folder}")


# step1_copy_and_prepare("INSAT_img_folder/Haryana_HR/HR_2022_TIR1_WV")


Step 1: Copied 3230 .tif files to INSAT_img_folder/Haryana_HR\INSATClean1_HR_2022


In [14]:
# import os
# import re

# def step2_timestamp_breakdown(clean_folder):
#     if not os.path.exists(clean_folder):
#         print(f"Clean folder does not exist: {clean_folder}")
#         return

#     tif_files = [f for f in os.listdir(clean_folder) if f.lower().endswith(".tif")]

#     round_count = 0
#     quarter_count = 0
#     irregular_count = 0
#     out_of_range_count = 0

#     for filename in tif_files:
#         match = re.search(r'_(\d{4})_', filename)  # Extract HHMM
#         if not match:
#             out_of_range_count += 1
#             continue

#         hhmm = match.group(1)
#         try:
#             minute = int(hhmm[2:])
#         except:
#             out_of_range_count += 1
#             continue

#         if minute in (0, 30):
#             round_count += 1
#         elif minute in (15, 45):
#             quarter_count += 1
#         elif 0 <= minute < 60:
#             irregular_count += 1
#         else:
#             out_of_range_count += 1

#     print(f"Step 2: Timestamp breakdown —")
#     print(f"  ➤ Round (00/30): {round_count}")
#     print(f"  ➤ Quarter (15/45): {quarter_count}")
#     print(f"  ➤ Irregular (others within 0–59): {irregular_count}")
#     print(f"  ➤ Out of range: {out_of_range_count}")

# # Example:
# step2_timestamp_breakdown("INSAT_img_folder/Haryana_HR/INSATClean1_HR_2022")


Step 2: Timestamp breakdown —
  ➤ Round (00/30): 0
  ➤ Quarter (15/45): 2902
  ➤ Irregular (others within 0–59): 328
  ➤ Out of range: 0


In [1]:
# import os
# import shutil
# from datetime import datetime, timedelta

# # === CONFIG ===
# source_folder = "INSAT_img_folder/Haryana_HR/INSATClean1_HR_2022"
# quarters_folder = "INSAT_img_folder/Haryana_HR/TQuarters_2022"
# rounds_folder = "INSAT_img_folder/Haryana_HR/TRounds_2022"
# outofrange_folder = "INSAT_img_folder/Haryana_HR/TOutofRange_2022"

# # Anchor minutes
# anchor_minutes = [0, 15, 30, 45]

# # Threshold for proximity (in minutes)
# proximity_threshold = 7

# # Valid time range
# valid_start = datetime.strptime("07:53", "%H:%M").time()
# valid_end = datetime.strptime("16:07", "%H:%M").time()

# # Counters
# count_quarters = 0
# count_rounds = 0
# count_outofrange = 0

# # === Helpers ===
# def extract_time_from_filename(filename):
#     try:
#         time_str = filename.split('_')[2]  # e.g. "1458"
#         hour = int(time_str[:2])
#         minute = int(time_str[2:])
#         return datetime.strptime(f"{hour}:{minute}", "%H:%M").time()
#     except:
#         return None

# def nearest_anchor(minute):
#     for anchor in anchor_minutes:
#         diff = abs(minute - anchor)
#         if diff > 30:  # handle wraparound, e.g., 58 close to 00
#             diff = 60 - diff
#         if 0 < diff <= proximity_threshold:  # exclude exact match
#             return anchor
#     return None

# # === Main Loop ===
# for file in os.listdir(source_folder):
#     if not file.endswith(".tif"):
#         continue

#     file_path = os.path.join(source_folder, file)
#     file_time = extract_time_from_filename(file)

#     if file_time is None:
#         print(f"⚠ Skipping malformed filename: {file}")
#         continue

#     # If time outside range and not exactly equal to the bounds
#     if file_time < valid_start or file_time > valid_end:
#         if file_time != valid_start and file_time != valid_end:
#             shutil.move(file_path, os.path.join(outofrange_folder, file))
#             print(f"⚠ {file} moved to TOutofRange (outside {valid_start}–{valid_end})")
#             count_outofrange += 1
#             continue

#     # Try to match anchor (only if NOT exactly 0, 15, 30, 45)
#     if file_time.minute in anchor_minutes:
#         # Skip exact 00, 15, 30, 45 (they belong in Clean folder, not here)
#         continue

#     anchor = nearest_anchor(file_time.minute)
#     if anchor is None:
#         shutil.move(file_path, os.path.join(outofrange_folder, file))
#         print(f"⚠ {file} not within ±7 min of 00/15/30/45 — moved to TOutofRange")
#         count_outofrange += 1
#         continue

#     # Route to folder based on proximity
#     if anchor in [15, 45]:
#         shutil.move(file_path, os.path.join(quarters_folder, file))
#         count_quarters += 1
#     elif anchor in [0, 30]:
#         shutil.move(file_path, os.path.join(rounds_folder, file))
#         count_rounds += 1

# # === Final Summary ===
# print("\n✅ Step 3: Irregulars moved —")
# print(f"  ➤ To TQuarters: {count_quarters}")
# print(f"  ➤ To TRounds: {count_rounds}")
# print(f"  ➤ To TOutofRange: {count_outofrange}")



✅ Step 3: Irregulars moved —
  ➤ To TQuarters: 64
  ➤ To TRounds: 264
  ➤ To TOutofRange: 0


In [4]:
import os
import re
from datetime import datetime, timedelta

# CONFIG
state_abbr = "HR"
year = 2022
expected_total = 2912
timestamp_tolerance = timedelta(minutes=7)

# Folders
clean_folder = f"INSAT_img_folder/Haryana_HR/INSATClean1_{state_abbr}_{year}"
quarters_folder = f"INSAT_img_folder/Haryana_HR/TQuarters_{year}"

# Time range (fixed from 07:53 to 16:07)
start_time = datetime.strptime(f"{year}-01-01 07:53", "%Y-%m-%d %H:%M")
end_time = datetime.strptime(f"{year}-01-01 16:07", "%Y-%m-%d %H:%M")
interval = timedelta(minutes=15)

# --- STEP 1: Generate all expected timestamps (2912 total)
expected_timestamps = []
while start_time <= end_time:
    for day_offset in range(1, 367):  # supports leap years
        try:
            ts = start_time.replace(month=1, day=1) + timedelta(days=day_offset - 1)
            expected_timestamps.append(ts.replace(hour=start_time.hour, minute=start_time.minute))
        except:
            continue
    start_time += interval

# --- STEP 2: Get timestamps from INSATClean1_HR_2022 folder
def extract_timestamp(filename):
    match = re.search(r"\d{8}_\d{4}", filename)
    if match:
        try:
            return datetime.strptime(match.group(), "%Y%m%d_%H%M")
        except:
            return None
    return None

actual_files = [f for f in os.listdir(clean_folder) if f.lower().endswith('.tif')]
actual_timestamps = set()
for fname in actual_files:
    ts = extract_timestamp(fname)
    if ts:
        actual_timestamps.add(ts)

# --- STEP 3: Identify missing timestamps
missing_timestamps = [ts for ts in expected_timestamps if ts not in actual_timestamps]

print(f"✅ Total files in {clean_folder}: {len(actual_timestamps)}")
print(f"❌ Missing timestamps (should be 10): {len(missing_timestamps)}")
print("---- MISSING TIMESTAMPS ----")
for ts in missing_timestamps:
    print(ts.strftime("%Y-%m-%d %H:%M"))

# --- STEP 4: Search in TQuarters_2022 for candidates within ±7 min of each missing
quarter_candidates = []

for filename in os.listdir(quarters_folder):
    if not filename.lower().endswith(".tif"):
        continue
    ts = extract_timestamp(filename)
    if not ts:
        continue
    for target_ts in missing_timestamps:
        if abs(ts - target_ts) <= timestamp_tolerance:
            rounded_minute = min([0, 15, 30, 45], key=lambda x: abs(ts.minute - x))
            corrected_ts = ts.replace(minute=rounded_minute, second=0)
            corrected_str = corrected_ts.strftime("%Y%m%d_%H%M")
            quarter_candidates.append({
                "original_file": os.path.join(quarters_folder, filename),
                "matched_missing_ts": target_ts,
                "corrected_ts": corrected_ts,
                "new_name": re.sub(r"\d{8}_\d{4}", corrected_str, filename)
            })
            break  # one match per file is enough

# --- STEP 5: Show candidate reintegrations (no move yet)
print("\n🔍 Potential reintegrations found in TQuarters_2022:")
for i, item in enumerate(quarter_candidates, 1):
    print(f"{i:02d}. {os.path.basename(item['original_file'])} ➜ corrected to {item['new_name']}")

print("\n❓ Do you want to move and rename these back into the clean folder? Type 'yes' to proceed.")



✅ Total files in INSAT_img_folder/Haryana_HR/INSATClean1_HR_2022: 0
❌ Missing timestamps (should be 10): 12078
---- MISSING TIMESTAMPS ----
2022-01-01 07:53
2022-01-02 07:53
2022-01-03 07:53
2022-01-04 07:53
2022-01-05 07:53
2022-01-06 07:53
2022-01-07 07:53
2022-01-08 07:53
2022-01-09 07:53
2022-01-10 07:53
2022-01-11 07:53
2022-01-12 07:53
2022-01-13 07:53
2022-01-14 07:53
2022-01-15 07:53
2022-01-16 07:53
2022-01-17 07:53
2022-01-18 07:53
2022-01-19 07:53
2022-01-20 07:53
2022-01-21 07:53
2022-01-22 07:53
2022-01-23 07:53
2022-01-24 07:53
2022-01-25 07:53
2022-01-26 07:53
2022-01-27 07:53
2022-01-28 07:53
2022-01-29 07:53
2022-01-30 07:53
2022-01-31 07:53
2022-02-01 07:53
2022-02-02 07:53
2022-02-03 07:53
2022-02-04 07:53
2022-02-05 07:53
2022-02-06 07:53
2022-02-07 07:53
2022-02-08 07:53
2022-02-09 07:53
2022-02-10 07:53
2022-02-11 07:53
2022-02-12 07:53
2022-02-13 07:53
2022-02-14 07:53
2022-02-15 07:53
2022-02-16 07:53
2022-02-17 07:53
2022-02-18 07:53
2022-02-19 07:53
2022-02-20

In [5]:
# Step 4
import os
from datetime import datetime, timedelta

# 📁 Step 0: Define paths
input_folder = r'INSAT_img_folder/Haryana_HR/INSATClean1_HR_2022'
log_folder = r'INSAT_img_folder\Haryana_HR'
os.makedirs(log_folder, exist_ok=True)
log_file_path = os.path.join(log_folder, 'missinglog.txt')

# 🕒 Step 1: Define expected times and bands
hour_blocks = [(f"{hour:02}15", f"{hour:02}45") for hour in range(8, 16)]
expected_times = [t for pair in hour_blocks for t in pair]
bands = ["TIR1", "WV"]

# 📅 Step 2: Set date range
start_date = datetime(2022, 9, 1)
end_date = datetime(2022, 11, 30)

# 📦 Step 3: Index all available files
available_files = {}

for file in os.listdir(input_folder):
    if file.endswith('.tif') and '_IMG_' in file:
        parts = file.split('_')
        try:
            date_str = parts[1]  # e.g., 01NOV2022
            time_str = parts[2]  # e.g., 0815
            band = parts[-1].replace('.tif', '')  # TIR1 or WV
            if len(date_str) == 9 and len(time_str) == 4:
                dt = datetime.strptime(date_str, '%d%b%Y')
                date_key = dt.strftime('%Y-%m-%d')
                key = (date_key, time_str, band)
                available_files[key] = file
        except Exception:
            continue  # Skip files with wrong format

# 📄 Step 4: Scan and log missing entries
with open(log_file_path, 'w') as log_file:
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        for time in expected_times:
            for band in bands:
                key = (date_str, time, band)
                if key not in available_files:
                    log_line = f"{date_str} {time} {band}\n"
                    log_file.write(log_line)
                    print(log_line.strip())
        current_date += timedelta(days=1)



2022-10-19 1145 TIR1
2022-10-19 1145 WV
2022-10-27 1015 TIR1
2022-10-27 1015 WV
2022-10-27 1045 TIR1
2022-10-27 1045 WV
2022-11-07 0845 TIR1
2022-11-07 0845 WV
2022-11-24 1415 TIR1
2022-11-24 1415 WV


In [ ]:
INSAT_img_folder/Haryana_HR/INSATClean1_HR_2022

In [12]:
import os
import re
from datetime import datetime, timedelta

# --- Paths ---
base_dir = r'INSAT_img_folder\Haryana_HR'
log_file = os.path.join(base_dir, 'missinglog_2022.txt')
search_dirs = [os.path.join(base_dir, 'TQuarters_2022'),
               os.path.join(base_dir, 'TRounds_2022')]

# --- Valid Bands ---
valid_bands = {"TIR1", "WV"}

# --- Helper to extract datetime and band from filename ---
def extract_dt_band(fname):
    match = re.search(r'_(\d{2}[A-Z]{3}\d{4})_(\d{4})_L1B_STD_V01R00_IMG_([A-Z0-9]+)\.tif', fname)
    if not match:
        return None, None
    date_str, time_str, band = match.groups()
    try:
        dt = datetime.strptime(date_str + time_str, "%d%b%Y%H%M")
        return dt, band
    except ValueError:
        return None, None

# --- Read missing log entries ---
with open(log_file, 'r') as f:
    missing_entries = [line.strip() for line in f if line.strip()]

# --- Scan for matches ---
for entry in missing_entries:
    try:
        date_str, time_str, band = entry.split()
        if band not in valid_bands:
            print(f"⚠️ Skipping unsupported band: {band}")
            continue
        target_dt = datetime.strptime(date_str + time_str, "%Y-%m-%d%H%M")
    except Exception as e:
        print(f"❌ Invalid entry format: {entry} — {e}")
        continue

    found = False
    for folder in search_dirs:
        for fname in os.listdir(folder):
            file_dt, file_band = extract_dt_band(fname)
            if not file_dt or not file_band:
                continue
            if file_band != band:
                continue
            # Match on same date + hour, with ±10 minute window
            if file_dt.date() == target_dt.date() and file_dt.hour == target_dt.hour:
                delta = abs((file_dt - target_dt).total_seconds()) / 60
                if delta <= 10:
                    print(f"🔎 Match found: {entry} ↔ {fname}")
                    found = True
    if not found:
        print(f"🚫 No match for: {entry}")


🚫 No match for: 2022-10-19 1145 TIR1
🚫 No match for: 2022-10-19 1145 WV
🚫 No match for: 2022-10-27 1015 TIR1
🚫 No match for: 2022-10-27 1015 WV
🚫 No match for: 2022-10-27 1045 TIR1
🚫 No match for: 2022-10-27 1045 WV
🚫 No match for: 2022-11-07 0845 TIR1
🚫 No match for: 2022-11-07 0845 WV
🚫 No match for: 2022-11-24 1415 TIR1
🚫 No match for: 2022-11-24 1415 WV


In [13]:
import os
from datetime import datetime, timedelta

# Paths
base_path = r'INSAT_img_folder\Haryana_HR'
missing_log_path = os.path.join(base_path, 'missinglog_2022.txt')
quarters_path = os.path.join(base_path, 'TQuarters_2022')
rounds_path = os.path.join(base_path, 'TRounds_2022')

# Load missing log
with open(missing_log_path, 'r') as f:
    missing_entries = [line.strip() for line in f if line.strip()]

# Parse function
def parse_filename(filename):
    try:
        parts = filename.split('_')
        date_part = parts[1]
        time_part = parts[2]
        band = parts[-1]
        dt = datetime.strptime(date_part + time_part, "%d%b%Y%H%M")
        return dt, band
    except:
        return None, None

# Build a list of all available image timestamps
search_dirs = [quarters_path, rounds_path]
all_files_info = []

for folder in search_dirs:
    for fname in os.listdir(folder):
        fpath = os.path.join(folder, fname)
        dt, band = parse_filename(fname)
        if dt and band:
            all_files_info.append((dt, band, fname, folder))

# Tolerance setting
tolerance_minutes = 20
found_count = 0

# Matching process
for entry in missing_entries:
    try:
        dt_str, band = entry.rsplit(' ', 1)
        target_dt = datetime.strptime(dt_str, "%Y-%m-%d %H%M")
    except ValueError:
        print(f"⚠️ Could not parse: {entry}")
        continue

    matched = False
    for dt, file_band, fname, folder in all_files_info:
        if band != file_band:
            continue
        if dt.date() == target_dt.date() and dt.hour == target_dt.hour:
            minute_diff = abs((dt - target_dt).total_seconds() / 60)
            if minute_diff <= tolerance_minutes:
                print(f"✅ Match found: {fname} in {os.path.basename(folder)} (Δ{int(minute_diff)} mins)")
                found_count += 1
                matched = True
                break

    if not matched:
        print(f"🚫 No match for: {entry}")

print(f"\n🔍 Done. Total matches found: {found_count}")


🚫 No match for: 2022-10-19 1145 TIR1
🚫 No match for: 2022-10-19 1145 WV
🚫 No match for: 2022-10-27 1015 TIR1
🚫 No match for: 2022-10-27 1015 WV
🚫 No match for: 2022-10-27 1045 TIR1
🚫 No match for: 2022-10-27 1045 WV
🚫 No match for: 2022-11-07 0845 TIR1
🚫 No match for: 2022-11-07 0845 WV
🚫 No match for: 2022-11-24 1415 TIR1
🚫 No match for: 2022-11-24 1415 WV

🔍 Done. Total matches found: 0


In [14]:
import os
import re
from datetime import datetime, timedelta

# Folder path for raw images
raw_folder = r"INSAT_img_folder\Haryana_HR\HR_2022_TIR1_WV"
missing_log_path = r"INSAT_img_folder\Haryana_HR\missinglog_2022.txt"

# Read all filenames in raw folder
raw_filenames = os.listdir(raw_folder)

# Extract metadata from filenames
raw_info = []
for file in raw_filenames:
    match = re.search(r"(\d{2})([A-Z]{3})(\d{4})_(\d{4}).*_(TIR1|WV)", file)
    if match:
        day, mon_str, year, time_str, band = match.groups()
        try:
            dt = datetime.strptime(f"{day}{mon_str}{year}_{time_str}", "%d%b%Y_%H%M")
            raw_info.append((dt, band, file))
        except:
            continue

# Read the missing log
with open(missing_log_path, 'r') as f:
    missing_entries = [line.strip() for line in f if line.strip()]

# Search with ±7-minute tolerance
tolerance = timedelta(minutes=7)
match_count = 0

for entry in missing_entries:
    try:
        date_str, time_str, band = entry.split()
        target_dt = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H%M")

        found = False
        for dt, b, fname in raw_info:
            if b == band and abs(dt - target_dt) <= tolerance:
                print(f"✅ Found match for: {entry} → File: {fname}")
                found = True
                match_count += 1
                break

        if not found:
            print(f"🚫 No match for: {entry}")

    except Exception as e:
        print(f"⚠️ Error processing: {entry} → {e}")

print(f"\n🔍 Done. Total matches found: {match_count} / {len(missing_entries)}")


🚫 No match for: 2022-10-19 1145 TIR1
🚫 No match for: 2022-10-19 1145 WV
🚫 No match for: 2022-10-27 1015 TIR1
🚫 No match for: 2022-10-27 1015 WV
🚫 No match for: 2022-10-27 1045 TIR1
🚫 No match for: 2022-10-27 1045 WV
🚫 No match for: 2022-11-07 0845 TIR1
🚫 No match for: 2022-11-07 0845 WV
🚫 No match for: 2022-11-24 1415 TIR1
🚫 No match for: 2022-11-24 1415 WV

🔍 Done. Total matches found: 0 / 10


In [15]:
import os
from datetime import datetime, timedelta

# 📁 Folder paths (you can modify as needed)
clean_folder = 'INSAT_img_folder/Haryana_HR/INSATClean1_HR_2021'
log_path = 'INSAT_img_folder/Haryana_HR/missinglog_2021.txt'

# 🕒 Expected half-hour time blocks (15 and 45 mins)
expected_times = [f"{h:02}15" for h in range(8, 16)] + [f"{h:02}45" for h in range(8, 16)]
expected_times.sort()
bands = ['TIR1', 'WV']

# 📅 Expected dates: 1 Sep to 30 Nov 2021
start_date = datetime(2021, 9, 1)
end_date = datetime(2021, 11, 30)

# 📦 Index existing files
available = set()

for file in os.listdir(clean_folder):
    if not file.endswith('.tif') or '_IMG_' not in file:
        continue
    parts = file.split('_')
    try:
        date_str = parts[1]       # e.g., 01NOV2021
        time_str = parts[2]       # e.g., 0815
        band = parts[-1].replace('.tif', '')  # TIR1 or WV
        dt = datetime.strptime(date_str, '%d%b%Y')
        key = (dt.strftime('%Y-%m-%d'), time_str, band)
        available.add(key)
    except Exception:
        continue

# 📝 Write missing entries
with open(log_path, 'w') as log_file:
    current = start_date
    while current <= end_date:
        date_str = current.strftime('%Y-%m-%d')
        for t in expected_times:
            for b in bands:
                if (date_str, t, b) not in available:
                    log_file.write(f"{date_str} {t} {b}\n")
        current += timedelta(days=1)

print(f"✅ Reconstructed log saved to: {log_path}")


✅ Reconstructed log saved to: INSAT_img_folder/Haryana_HR/missinglog_2021.txt


In [17]:
import os
from datetime import datetime, timedelta
import shutil

# Base folder structure
base_folder = 'INSAT_img_folder/Haryana_HR'
clean_folder = os.path.join(base_folder, 'INSATClean1_HR_2021')
rounds_folder = os.path.join(base_folder, 'TRounds_2021')
quarters_folder = os.path.join(base_folder, 'TQuarters_2021')
log_path = os.path.join(base_folder, 'missinglog_2021.txt')
recovery_log = os.path.join(base_folder, 'recoverylog_2021.txt')

# Read missing timestamps from log file
with open(log_path, 'r', encoding='utf-8') as f:
    missing_entries = [line.strip().split() for line in f if line.strip()]

# Index all files in TRounds and TQuarters
def index_files(folder):
    file_dict = {}
    for file in os.listdir(folder):
        if file.endswith('.tif') and '_IMG_' in file:
            parts = file.split('_')
            try:
                date_str = parts[1]
                time_str = parts[2]
                band = parts[-1].replace('.tif', '')
                dt = datetime.strptime(date_str + time_str, '%d%b%Y%H%M')
                file_dict[(dt, band)] = os.path.join(folder, file)
            except:
                continue
    return file_dict

rounds_index = index_files(rounds_folder)
quarters_index = index_files(quarters_folder)

# Combine both indexes
all_index = {**rounds_index, **quarters_index}

# Recovery loop with ±7 min tolerance
found_count = 0
with open(recovery_log, 'w', encoding='utf-8') as log:
    for date_str, time_str, band in missing_entries:
        target_time = datetime.strptime(date_str + time_str, '%Y-%m-%d%H%M')
        found = False

        for dt_offset in range(-7, 8):
            candidate_time = target_time + timedelta(minutes=dt_offset)
            key = (candidate_time, band)
            if key in all_index:
                src = all_index[key]
                dst_filename = f"3RIMG_{candidate_time.strftime('%d%b%Y').upper()}_{candidate_time.strftime('%H%M')}_L1B_STD_V01R00_IMG_{band}.tif"
                dst_path = os.path.join(clean_folder, dst_filename)
                shutil.copy2(src, dst_path)
                log.write(f"Recovered: {dst_filename} from {os.path.basename(src)}\n")
                found_count += 1
                found = True
                break

        if not found:
            log.write(f"Not Found: {date_str} {time_str} {band}\n")

print(f"Step 6 Part 1 complete. Total recovered: {found_count}")
print(f"Recovery log saved to: {recovery_log}")


Step 6 Part 1 complete. Total recovered: 8
Recovery log saved to: INSAT_img_folder/Haryana_HR\recoverylog_2021.txt


In [20]:
import os
import shutil

# Base folder where everything lives
base_folder = r'INSAT_img_folder/Haryana_HR'

# Subfolders
clean_folder = os.path.join(base_folder, 'INSATClean1_HR_2021')
quarters_folder = os.path.join(base_folder, 'TQuarters_2021')
rounds_folder = os.path.join(base_folder, 'TRounds_2021')

# Log file from recovery step
recovery_log_path = os.path.join(base_folder, 'recoverylog_2021.txt')

# Candidate source folders to try
source_folders = [quarters_folder, rounds_folder]

# Reversal starts
with open(recovery_log_path, 'r') as log:
    for line in log:
        line = line.strip()
        if line.startswith("Recovered:"):
            try:
                filename = line.split("Recovered:")[1].split("from")[0].strip()
                moved_file_path = os.path.join(clean_folder, filename)
                
                # Try to move back to the correct original folder
                restored = False
                for src_folder in source_folders:
                    if os.path.exists(src_folder):
                        dst_path = os.path.join(src_folder, filename)
                        if os.path.exists(moved_file_path):
                            shutil.move(moved_file_path, dst_path)
                            print(f"✅ Reversed: {filename} → {src_folder}")
                            restored = True
                            break
                if not restored:
                    print(f"⚠️ Could not restore: {filename} (Original folder not known)")
            except Exception as e:
                print(f"❌ Error processing line: {line}\n{e}")



✅ Reversed: 3RIMG_25SEP2021_0849_L1B_STD_V01R00_IMG_TIR1.tif → INSAT_img_folder/Haryana_HR\TQuarters_2021
✅ Reversed: 3RIMG_25SEP2021_0849_L1B_STD_V01R00_IMG_WV.tif → INSAT_img_folder/Haryana_HR\TQuarters_2021
✅ Reversed: 3RIMG_25SEP2021_0949_L1B_STD_V01R00_IMG_TIR1.tif → INSAT_img_folder/Haryana_HR\TQuarters_2021
✅ Reversed: 3RIMG_25SEP2021_0949_L1B_STD_V01R00_IMG_WV.tif → INSAT_img_folder/Haryana_HR\TQuarters_2021
✅ Reversed: 3RIMG_26SEP2021_1049_L1B_STD_V01R00_IMG_TIR1.tif → INSAT_img_folder/Haryana_HR\TQuarters_2021
✅ Reversed: 3RIMG_26SEP2021_1049_L1B_STD_V01R00_IMG_WV.tif → INSAT_img_folder/Haryana_HR\TQuarters_2021
✅ Reversed: 3RIMG_26SEP2021_1349_L1B_STD_V01R00_IMG_TIR1.tif → INSAT_img_folder/Haryana_HR\TQuarters_2021
✅ Reversed: 3RIMG_26SEP2021_1349_L1B_STD_V01R00_IMG_WV.tif → INSAT_img_folder/Haryana_HR\TQuarters_2021


In [21]:
# import os
# from datetime import datetime, timedelta

# # === Paths ===
# base_folder = 'INSAT_img_folder/Haryana_HR'
# main_folder = os.path.join(base_folder, 'INSATClean1_HR_2021')
# log_file = os.path.join(base_folder, 'missinglog_2021.txt')
# recovery_log = os.path.join(base_folder, 'recoverylog_2021.txt')
# search_folders = [
#     os.path.join(base_folder, 'TRounds_2021'),
#     os.path.join(base_folder, 'TQuarters_2021')
# ]

# # === Settings ===
# tolerance_minutes = 20

# # === Load missing entries ===
# with open(log_file, 'r') as f:
#     missing_entries = [line.strip().split() for line in f if line.strip()]

# # === Recovery Log Rewrite ===
# recovered = 0
# log_lines = []

# for date_str, time_str, band in missing_entries:
#     target_dt = datetime.strptime(date_str + time_str, "%Y-%m-%d%H%M")
#     found = False

#     for folder in search_folders:
#         for file in os.listdir(folder):
#             if file.endswith('.tif') and f'_IMG_{band}' in file:
#                 try:
#                     parts = file.split('_')
#                     f_dt = datetime.strptime(parts[1] + parts[2], "%d%b%Y%H%M")
#                     time_diff = abs((f_dt - target_dt).total_seconds()) / 60
#                     if time_diff <= tolerance_minutes:
#                         log_lines.append(f"Recovered: {file} from {folder}")
#                         recovered += 1
#                         found = True
#                         break
#                 except Exception:
#                     continue
#         if found:
#             break
#     if not found:
#         log_lines.append(f"Not Found: {date_str} {time_str} {band}")

# # === Write the new recovery log ===
# with open(recovery_log, 'w') as f:
#     f.write("\n".join(log_lines))

# print(f"✅ Recovery log rewritten. Total recovered: {recovered}")
# print(f"📄 Saved to: {recovery_log}")


✅ Recovery log rewritten. Total recovered: 28
📄 Saved to: INSAT_img_folder/Haryana_HR\recoverylog_2021.txt


In [23]:
import os
import pandas as pd
from datetime import datetime

# --- CONFIGURATION ---
stations_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\cleaned_station_data"  # folder with 25 station files (2021-2022)
raw_2023_folder = r"C:\Users\admin\Documents\Hackathon_dataset_2"       # folder with raw CPCB files (2023)
expected_rows = 8760                             # 365 days * 24 hours

# --- STEP 1: Get base names from 25 station files ---
station_files = [f for f in os.listdir(stations_folder) if f.endswith(".csv")]
station_bases = [os.path.splitext(f)[0].split("_C")[0] for f in station_files]  # remove suffix like _C1

# --- STEP 2: Scan raw folder for matching files ---
raw_files = [f for f in os.listdir(raw_2023_folder) if f.endswith(".csv")]

# --- STEP 3: Initialize result containers ---
complete_2023 = []
partial_2023 = []
missing_or_invalid = []

# --- STEP 4: Match and validate each station ---
for station_base in station_bases:
    matched_file = None

    for raw_file in raw_files:
        raw_base = os.path.splitext(raw_file)[0]
        if raw_base.startswith(station_base):
            matched_file = os.path.join(raw_2023_folder, raw_file)
            break

    if matched_file is None:
        missing_or_invalid.append((station_base, "No matching raw file found"))
        continue

    try:
        df = pd.read_csv(matched_file)
        if df.empty:
            missing_or_invalid.append((station_base, "File is empty"))
            continue

        date_col = df.columns[0]
        first_dt = pd.to_datetime(df.iloc[0, 0], errors='coerce')
        last_dt = pd.to_datetime(df.iloc[-1, 0], errors='coerce')

        if pd.isna(first_dt) or pd.isna(last_dt):
            missing_or_invalid.append((station_base, "Datetime parsing failed"))
            continue

        if first_dt <= datetime(2023, 1, 1, 0, 0) and last_dt >= datetime(2023, 12, 31, 23, 0):
            if len(df) >= expected_rows:
                complete_2023.append(station_base)
            else:
                partial_2023.append((station_base, len(df)))
        else:
            partial_2023.append((station_base, len(df)))

    except Exception as e:
        missing_or_invalid.append((station_base, f"Error reading file: {str(e)}"))

# --- STEP 5: Print results ---
print("\n=== ✅ Stations with COMPLETE 2023 Data (8760 rows) ===")
for s in complete_2023:
    print(s)

print("\n=== ⚠️ Stations with PARTIAL 2023 Data ===")
for s, count in partial_2023:
    print(f"{s} --> {count} rows")

print("\n=== ❌ Missing or Invalid Raw Data ===")
for s, reason in missing_or_invalid:
    print(f"{s} --> {reason}")



=== ✅ Stations with COMPLETE 2023 Data (8760 rows) ===

=== ⚠️ Stations with PARTIAL 2023 Data ===
DL009 --> 113168 rows
DL011 --> 109905 rows
DL022 --> 45230 rows
DL024 --> 45230 rows
DL029 --> 44568 rows
HR001 --> 109992 rows
HR002 --> 64096 rows
HR003 --> 64272 rows
HR004 --> 63519 rows
HR009 --> 36999 rows
KA001 --> 116050 rows
KA002 --> 116112 rows
KA007 --> 41771 rows
KA015 --> 32438 rows
KA018 --> 30924 rows
MH004 --> 67944 rows
MH007 --> 57881 rows
MH012 --> 33183 rows
MH026 --> 20916 rows
MH033 --> 20843 rows

=== ❌ Missing or Invalid Raw Data ===


In [24]:
import os
import shutil

# --- INPUT FOLDERS ---
stations_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\cleaned_station_data"
raw_2023_folder = r"C:\Users\admin\Documents\Hackathon_dataset_2"

# --- OUTPUT FOLDER ---
matched_output_folder = os.path.join(r"C:\Users\admin\Desktop\cleaned7\cleaned7", "Matched_Raw_2023")
os.makedirs(matched_output_folder, exist_ok=True)

# --- STEP 1: Extract base names from 20 station files ---
station_files = [f for f in os.listdir(stations_folder) if f.endswith('.csv')]
station_bases = [os.path.splitext(f)[0].split("_C")[0] for f in station_files]  # removes _C1, _C2 etc.

# --- STEP 2: Get all raw files ---
raw_files = [f for f in os.listdir(raw_2023_folder) if f.endswith('.csv')]

# --- STEP 3: Match and copy ---
matched_count = 0
for station_base in station_bases:
    for raw_file in raw_files:
        if raw_file.startswith(station_base):
            src = os.path.join(raw_2023_folder, raw_file)
            dest = os.path.join(matched_output_folder, raw_file)
            shutil.copy2(src, dest)
            matched_count += 1
            break  # match only the first that startswith

print(f"\n✅ Total matched and copied files: {matched_count}")
print(f"📂 Files copied to: {matched_output_folder}")



✅ Total matched and copied files: 20
📂 Files copied to: C:\Users\admin\Desktop\cleaned7\cleaned7\Matched_Raw_2023


In [26]:
import os
import pandas as pd

# --- CONFIGURATION ---
raw_2023_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\Matched_Raw_2023"

# --- STEP 1: Get all .csv files in the raw folder ---
csv_files = [f for f in os.listdir(raw_2023_folder) if f.endswith('.csv')]

# --- STEP 2: Loop through and print last row (first 2 columns only) ---
for file in csv_files:
    file_path = os.path.join(raw_2023_folder, file)
    try:
        df = pd.read_csv(file_path)
        if df.empty:
            print(f"\n{file} --> File is empty")
            continue

        # Get last row, then select only first 2 columns
        last_row = df.tail(1).iloc[:, :2]

        print(f"\n=== Last row of: {file} (First 2 columns only) ===")
        print(last_row.to_string(index=False))

    except Exception as e:
        print(f"\n{file} --> Error reading file: {str(e)}")




=== Last row of: DL009.csv (First 2 columns only) ===
          From Date             To Date
2023-03-31 23:00:00 2023-04-01 00:00:00

=== Last row of: DL011.csv (First 2 columns only) ===
          From Date             To Date
2023-03-31 23:00:00 2023-04-01 00:00:00

=== Last row of: DL022.csv (First 2 columns only) ===
          From Date             To Date
2023-03-31 23:00:00 2023-04-01 00:00:00

=== Last row of: DL024.csv (First 2 columns only) ===
          From Date             To Date
2023-03-31 23:00:00 2023-04-01 00:00:00

=== Last row of: DL029.csv (First 2 columns only) ===
          From Date             To Date
2023-03-31 23:00:00 2023-04-01 00:00:00

=== Last row of: HR001.csv (First 2 columns only) ===
          From Date             To Date
2023-03-31 23:00:00 2023-04-01 00:00:00

=== Last row of: HR002.csv (First 2 columns only) ===
          From Date             To Date
2023-03-31 23:00:00 2023-04-01 00:00:00

=== Last row of: HR003.csv (First 2 columns only) ===


In [2]:
import os
import pandas as pd

# Paths
base_path = r"C:\Users\admin\Desktop\cleaned7\cleaned7"
raw_folder = os.path.join(base_path, "Matched_Raw_2023")
cleaned_folder = os.path.join(base_path, "cleaned_station_data")
output_folder = os.path.join(base_path, "Matched_Raw_2023_Filtered")
os.makedirs(output_folder, exist_ok=True)

# Date/time constraints
start_date = pd.Timestamp("2022-12-01 00:00")
end_date = pd.Timestamp("2023-03-31 23:00")

def filter_datetime(df, datetime_col):
    """Filters the dataframe based on date/time conditions."""
    dt = pd.to_datetime(df[datetime_col], format="%Y-%m-%d %H:%M:%S", errors='coerce')
    mask = (
        (dt >= start_date) &
        (dt <= end_date) &
        (dt.dt.hour >= 8) &
        (dt.dt.hour <= 16)
    )
    return df.loc[mask]


# Process each raw file
for raw_file in os.listdir(raw_folder):
    if not raw_file.endswith(".csv"):
        continue

    raw_path = os.path.join(raw_folder, raw_file)
    cleaned_path = os.path.join(cleaned_folder, raw_file)

    # Skip if there's no matching cleaned file
    if not os.path.exists(cleaned_path):
        print(f"⚠️ No cleaned match found for {raw_file}, skipping.")
        continue

    # Read files
    df_raw = pd.read_csv(raw_path)
    df_cleaned = pd.read_csv(cleaned_path, nrows=0)  # only columns

    # Get reference columns from cleaned file
    ref_cols = [c for c in df_raw.columns if c in df_cleaned.columns]

    # Keep only matching columns
    df_filtered = df_raw[ref_cols].copy()

    # Detect first column name (datetime col)
    datetime_col = ref_cols[0]

    # Apply datetime filter
    df_filtered = filter_datetime(df_filtered, datetime_col)

    # Save filtered file
    output_path = os.path.join(output_folder, raw_file)
    df_filtered.to_csv(output_path, index=False)

print("✅ Matched_Raw_2023_Filtered has been rebuilt with file-specific column matching and time filtering.")


✅ Matched_Raw_2023_Filtered has been rebuilt with file-specific column matching and time filtering.


In [6]:

import os
import pandas as pd

clean_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\cleaned_station_data"
filtered_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\Matched_Raw_2023_Filtered"

mismatches = []

for clean_file in os.listdir(clean_folder):
    clean_path = os.path.join(clean_folder, clean_file)
    filtered_path = os.path.join(filtered_folder, clean_file)

    if os.path.exists(filtered_path):
        clean_df = pd.read_csv(clean_path, nrows=0)  # just headers
        filtered_df = pd.read_csv(filtered_path, nrows=0)

        if list(clean_df.columns) != list(filtered_df.columns):
            mismatches.append({
                "file": clean_file,
                "clean_cols": list(clean_df.columns),
                "filtered_cols": list(filtered_df.columns)
            })

if mismatches:
    print("⚠️ Column mismatches found:")
    for m in mismatches:
        print(f"\nFile: {m['file']}")
        print(f"Clean columns:    {m['clean_cols']}")
        print(f"Filtered columns: {m['filtered_cols']}")
else:
    print("✅ All files match in column names and order.")


⚠️ Column mismatches found:

File: DL009.csv
Clean columns:    ['From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'Temp (degree C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'AT (degree C)']
Filtered columns: ['From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'AT (degree C)']

File: DL011.csv
Clean columns:    ['From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'Temp (degree C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'AT (degree C)']
Filtered columns: ['From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'AT (degree C)']

File: DL022.csv
Clean colum

In [8]:
import os
import pandas as pd

filtered_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\Matched_Raw_2023_Filtered"
clean_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\cleaned_station_data"

mismatch_report = []

for filtered_file in os.listdir(filtered_folder):
    if filtered_file.endswith(".csv"):
        filtered_path = os.path.join(filtered_folder, filtered_file)
        clean_path = os.path.join(clean_folder, filtered_file)
        
        if not os.path.exists(clean_path):
            mismatch_report.append((filtered_file, "Missing in clean_station_data"))
            continue
        
        df_filtered = pd.read_csv(filtered_path)
        df_clean = pd.read_csv(clean_path)
        
        # Compare columns
        cols_filtered = list(df_filtered.columns)
        cols_clean = list(df_clean.columns)
        
        if cols_filtered != cols_clean:
            mismatch_report.append((filtered_file, "Column mismatch", cols_filtered, cols_clean))
        
        # Compare row count
        if len(df_filtered) != len(df_clean):
            mismatch_report.append((filtered_file, "Row count mismatch", len(df_filtered), len(df_clean)))

if mismatch_report:
    print("⚠️ Mismatches found:")
    for item in mismatch_report:
        print(item)
else:
    print("✅ All files in Matched_Raw_2023_Filtered match columns and row counts with clean_station_data.")


⚠️ Mismatches found:
('DL009.csv', 'Column mismatch', ['From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'AT (degree C)'], ['From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'Temp (degree C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'AT (degree C)'])
('DL009.csv', 'Row count mismatch', 1089, 6570)
('DL011.csv', 'Column mismatch', ['From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'AT (degree C)'], ['From Date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'Temp (degree C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'SR (W/mt2)', 'AT (degree C)'])
('DL011.csv', 'Row coun

In [10]:
import os
import pandas as pd

filtered_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\Matched_Raw_2023_Filtered"
clean_folder = r"C:\Users\admin\Desktop\cleaned7\cleaned7\cleaned_station_data"

mismatch_report = []

for filtered_file in os.listdir(filtered_folder):
    if filtered_file.endswith(".csv"):
        filtered_path = os.path.join(filtered_folder, filtered_file)
        clean_path = os.path.join(clean_folder, filtered_file)
        
        if not os.path.exists(clean_path):
            mismatch_report.append((filtered_file, "❌ Missing in clean_station_data"))
            continue
        
        df_filtered = pd.read_csv(filtered_path)
        df_clean = pd.read_csv(clean_path)
        
        cols_filtered = list(df_filtered.columns)
        cols_clean = list(df_clean.columns)
        
        if cols_filtered != cols_clean:
            extra_in_filtered = [c for c in cols_filtered if c not in cols_clean]
            missing_in_filtered = [c for c in cols_clean if c not in cols_filtered]
            
            mismatch_report.append((
                filtered_file,
                "⚠️ Column mismatch",
                {"Extra in filtered": extra_in_filtered, "Missing in filtered": missing_in_filtered}
            ))
        
        if len(df_filtered) != len(df_clean):
            mismatch_report.append((
                filtered_file,
                "⚠️ Row count mismatch",
                {"Filtered rows": len(df_filtered), "Clean rows": len(df_clean)}
            ))

if mismatch_report:
    print("\n--- MISMATCH REPORT ---")
    for entry in mismatch_report:
        print(entry)
else:
    print("✅ All files in Matched_Raw_2023_Filtered match columns and row counts with clean_station_data.")



--- MISMATCH REPORT ---
('DL009.csv', '⚠️ Column mismatch', {'Extra in filtered': [], 'Missing in filtered': ['NH3 (ug/m3)', 'SO2 (ug/m3)', 'Temp (degree C)']})
('DL009.csv', '⚠️ Row count mismatch', {'Filtered rows': 1089, 'Clean rows': 6570})
('DL011.csv', '⚠️ Column mismatch', {'Extra in filtered': [], 'Missing in filtered': ['NH3 (ug/m3)', 'SO2 (ug/m3)', 'Temp (degree C)']})
('DL011.csv', '⚠️ Row count mismatch', {'Filtered rows': 1089, 'Clean rows': 6570})
('DL022.csv', '⚠️ Column mismatch', {'Extra in filtered': [], 'Missing in filtered': ['WD (deg)']})
('DL022.csv', '⚠️ Row count mismatch', {'Filtered rows': 1089, 'Clean rows': 6570})
('DL024.csv', '⚠️ Column mismatch', {'Extra in filtered': [], 'Missing in filtered': ['WD (deg)']})
('DL024.csv', '⚠️ Row count mismatch', {'Filtered rows': 1089, 'Clean rows': 6570})
('DL029.csv', '⚠️ Column mismatch', {'Extra in filtered': [], 'Missing in filtered': ['WD (deg)']})
('DL029.csv', '⚠️ Row count mismatch', {'Filtered rows': 1089, 'C

In [12]:
import os
import pandas as pd

# ------------------- CONFIG -------------------
raw_folder = r"C:\Users\admin\Documents\Hackathon_dataset_2"
station_list_file = r"C:\Users\admin\Desktop\cleaned7\cleaned7\cleaned_station_data"
output_folder = r"C:\Users\admin\Desktop\cpcb_cleaned_output\Cleaned7"
os.makedirs(output_folder, exist_ok=True)

# Required columns in final output
required_columns = [
    "From Date", "PM2.5 (ug/m3)", "PM10 (ug/m3)", "NO (ug/m3)", "NO2 (ug/m3)",
    "NOx (ppb)", "NH3 (ug/m3)", "SO2 (ug/m3)", "CO (mg/m3)", "Ozone (ug/m3)",
    "Temp (degree C)", "RH (%)", "WS (m/s)", "WD (deg)", "SR (W/mt2)", "AT (degree C)"
]

# ------------------- STEP 1: Get 20 station filenames -------------------
station_files = {f for f in os.listdir(station_list_file) if f.lower().endswith(".csv")}

# ------------------- STEP 2: Process matching raw files -------------------
for filename in os.listdir(raw_folder):
    if filename in station_files:
        file_path = os.path.join(raw_folder, filename)

        try:
            # Read file (skip first 2 rows, keep as strings)
            df = pd.read_csv(file_path, skiprows=2, dtype=str)

            # Strip column names of spaces
            df.columns = df.columns.str.strip()

            # Function to parse date string
            def parse_datetime(val):
                val = str(val).strip()
                for fmt in ("%Y-%m-%d %H:%M", "%d-%m-%Y %H:%M"):
                    try:
                        return pd.to_datetime(val, format=fmt)
                    except ValueError:
                        continue
                return pd.NaT

            # Convert From Date column
            if "From Date" not in df.columns:
                raise ValueError("Missing 'From Date' column")

            df["From Date"] = df["From Date"].apply(parse_datetime)
            df = df.dropna(subset=["From Date"])

            # Filter year between 2021 and 2022
            df = df[(df["From Date"].dt.year >= 2021) & (df["From Date"].dt.year <= 2022)]

            # Filter time range 08:00–16:00
            df = df[(df["From Date"].dt.hour >= 8) & (df["From Date"].dt.hour <= 16)]

            # Convert numeric columns where possible
            for col in df.columns:
                if col != "From Date":
                    df[col] = pd.to_numeric(df[col], errors="coerce")

            # Keep only required columns that exist
            available_cols = [col for col in required_columns if col in df.columns]
            df = df[available_cols]

            # Save cleaned file
            output_path = os.path.join(output_folder, filename)
            df.to_csv(output_path, index=False)

            print(f"✅ Cleaned: {filename} | Rows: {len(df)}")

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")


❌ Error processing DL009.csv: Missing 'From Date' column
❌ Error processing DL011.csv: Missing 'From Date' column
❌ Error processing DL022.csv: Missing 'From Date' column
❌ Error processing DL024.csv: Missing 'From Date' column
❌ Error processing DL029.csv: Missing 'From Date' column
❌ Error processing HR001.csv: Missing 'From Date' column
❌ Error processing HR002.csv: Missing 'From Date' column
❌ Error processing HR003.csv: Missing 'From Date' column
❌ Error processing HR004.csv: Missing 'From Date' column
❌ Error processing HR009.csv: Missing 'From Date' column
❌ Error processing KA001.csv: Missing 'From Date' column
❌ Error processing KA002.csv: Missing 'From Date' column
❌ Error processing KA007.csv: Missing 'From Date' column
❌ Error processing KA015.csv: Missing 'From Date' column
❌ Error processing KA018.csv: Missing 'From Date' column
❌ Error processing MH004.csv: Missing 'From Date' column
❌ Error processing MH007.csv: Missing 'From Date' column
❌ Error processing MH012.csv: M